# 📊 PyFlow: Deep PYUSD Analysis with Google Cloud's Premium RPC Methods

**Hackathon Context:** This notebook is developed for the **PayPal x Google Cloud Web3 Bounty**, demonstrating how **Google Cloud Platform's Blockchain Node Engine** unlocks powerful, cost-effective analysis of the **PayPal USD (PYUSD)** stablecoin on Ethereum.

---

## The Challenge: Unlocking Deep Blockchain Insights

Understanding the intricate movements, internal logic, and real-world interactions of stablecoins like **PYUSD** often requires deep, computationally intensive blockchain analysis. Standard block explorers and basic RPC calls provide only surface-level data, while accessing advanced tracing and state inspection methods on most platforms is prohibitively expensive or heavily rate-limited due to their high computational demands.

## The Solution: PyFlow leveraging GCP's Advantage

This notebook, **PyFlow**, provides a comprehensive toolkit for advanced PYUSD analysis by specifically utilizing **Google Cloud Platform's premium RPC debug and trace methods**.

> **🚀 GCP's Unique Offering: Cost-Effective Access to High-Multiplier Methods**
>
> Many advanced RPC methods carry significant **request multipliers** due to their computational intensity. For example, a method with a `50x` multiplier consumes the equivalent quota/cost of 50 basic calls (like `eth_call`). Methods like `trace_replayTransaction` have an even higher `100x` multiplier.
>
> **GCP's Blockchain Node Engine stands out by offering generous free quotas even for these high-multiplier methods**, effectively democratizing access to capabilities previously reserved for specialized infrastructure or high budgets.

This allows PyFlow to perform analysis typically infeasible elsewhere, such as:

*   **Forensic Accounting:** Tracing PYUSD flow through complex multi-contract DeFi interactions using methods like `debug_traceTransaction` (`50x`).
*   **Gas Optimization Analysis:** Pinpointing exact gas costs within internal PYUSD functions or integrations.
*   **Security Investigations:** Replaying failed transactions (`trace_replayTransaction`, `100x`) or examining state changes (`stateDiff` via replay).
*   **Smart Contract Auditing:** Verifying internal logic, storage layout (`debug_storageRangeAt`, `50x`), and event emission (`eth_getLogs`, `50x`).
*   **Network Health Insights:** Analyzing pending transaction queues (`txpool_status`, `50x`) and estimating confirmation times.

## 🛠️ Methods Explored (with GCP Request Multipliers):

This notebook provides practical implementations and analysis using the following GCP-powered methods for PYUSD on Ethereum. Multipliers indicate the relative request cost compared to a standard call:

*   **Detailed Tracing:**
    *   `debug_traceTransaction` (`50x`): In-depth EVM execution trace (using `callTracer` & `structLog`).
    *   `trace_transaction` (`50x`): Alternative transaction tracing method.
*   **Block-Level Analysis:**
    *   `trace_block` (`50x`, Mainnet only): Trace all transactions within a specified block.
    *   `debug_traceBlockByNumber` / `debug_traceBlockByHash` (`50x`): Alternative block tracing.
*   **State Replay & Simulation:**
    *   `trace_replayTransaction` (`100x`, Mainnet only): Re-execute a past transaction with tracers.
    *   `trace_replayBlockTransactions` (`100x`, Mainnet only): Re-execute all transactions in a block with tracers.
    *   `trace_call` (`50x`, Mainnet only): Simulate transaction calls without sending to the network.
*   **State & Data Retrieval:**
    *   `eth_getLogs` (`50x`): Efficiently fetch specific PYUSD events (e.g., Transfers, Approvals).
    *   `eth_getCode` (`10x`): Retrieve deployed contract bytecode.
    *   `debug_storageRangeAt` (`50x`): Inspect raw contract storage slots.
    *   `eth_getProof` (`50x`): Fetch Merkle proofs for state verification.
*   **Network Monitoring:**
    *   `txpool_status` (`50x`): Analyze pending/queued transaction counts.

*(Note: Multipliers are based on GCP documentation and highlight the computational intensity absorbed by the service.)*

---

**💡 Goal:** By the end of this notebook, you will understand how to leverage GCP's unique RPC capabilities, including high-multiplier methods offered with generous quotas, to perform advanced, cost-effective blockchain intelligence specifically tailored for the PYUSD stablecoin.


## 🛠️ Environment Setup: Installing Dependencies for PyFlow
---

This cell installs the necessary Python packages to run the PyFlow analysis notebook. It sets up a complete environment for interacting with the Ethereum blockchain (via GCP), analyzing PYUSD data, generating visualizations, and connecting to Google Cloud services.

### 📊 Key Dependencies & Purpose:

| Category                 | Packages                                                       | Purpose                                                             |
| :----------------------- | :------------------------------------------------------------- | :------------------------------------------------------------------ |
| **Core Blockchain/Data** | `web3`, `pandas`, `numpy`                                    | Ethereum RPC interaction, data manipulation                         |
| **Visualization**        | `matplotlib`, `plotly`, `seaborn`, `networkx`, `graphviz`    | Charts, transaction graphs, visual analysis                         |
| **Google Cloud**         |  `gspread`, `oauth2client` | Accessing Google Sheets export, Authentication |
| **Ethereum Utilities**   | `eth-utils`, `rlp`, `tqdm`                                   | Cryptographic functions, RLP encoding, progress bars                |
| **Notebook Enhancement** | `ipywidgets`, `rich`                                         | Interactive controls, improved console output                     |

### ⚙️ Runtime Notes:

*   **Environment:** Designed primarily for Google Colab.
*   **Resources:** A standard Colab runtime is usually sufficient, but a High-RAM runtime is recommended for analyzing very large blocks or complex transaction traces. GPU is generally not required.
*   **Colab Features:** The setup automatically installs system-level `graphviz` and enables interactive data tables within Colab.

> **⏳ Installation Time:** The process uses `pip` and typically completes in **1-2 minutes**. Please ensure this cell executes successfully before proceeding.

In [ ]:
# =============================================================================================
# 🛠️ Environment Setup and Package Installation
# =============================================================================================
# This cell installs and configures all necessary packages for blockchain data analysis.
# The setup process may take 1-2 minutes to complete.

import sys
import subprocess
import time
from IPython.display import clear_output
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress, SpinnerColumn, TextColumn

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",          # Informational messages
    "success": "spring_green3", # Success indicators
    "warning": "gold3",       # Warning messages
    "error": "red3",          # Error messages
    "highlight": "royal_blue1"  # Highlighted information
})

# Create console with auto color system detection for better visual feedback
console = Console(theme=custom_theme)

console.print("\n✨ Environment Setup and Package Installation ✨", style="bold cyan3")
console.print("─────────────────────────────────────────────────", style="cyan3")

console.print("🔄 Starting package installation process...", style="info")

# Function to install packages and handle errors with better formatting
# This provides visual feedback during the installation process
def install_packages(packages, description):
    """Install specified packages with progress indicator and error handling"""
    with Progress(
        SpinnerColumn(),
        TextColumn(f"[info]Installing {description}..."),
        transient=True,
    ) as progress:
        task = progress.add_task("", total=None)
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages.split())
            console.print(f"✓ {description} installed", style="success")
            return True
        except subprocess.CalledProcessError:
            console.print(f"❌ Error installing {description}", style="error")
            return False

# =============================================================================================
# Core Libraries Installation
# =============================================================================================

# Install core data processing libraries
# - web3: For blockchain interaction and smart contract calls
# - pandas: For data manipulation and analysis
# - numpy: For numerical operations
# - matplotlib: For basic visualization
success = install_packages("web3==6.11.1 pandas numpy matplotlib",
                         "Core data libraries (web3, pandas, numpy, matplotlib)")

# Install advanced visualization and analysis libraries
# - plotly: For interactive charts
# - seaborn: For statistical visualizations
# - networkx: For blockchain transaction network analysis
if success:
    success = install_packages("plotly seaborn networkx",
                             "Visualization and analysis libraries (plotly, seaborn, networkx)")

# Install visualization export libraries
# - kaleido: For high-quality Plotly chart exports to PNG/PDF/SVG
if success:
    success = install_packages("kaleido",
                             "Visualization export library (required for exporting charts to images)")

# Install Google API libraries for data access and storage
# - gspread: For Google Sheets integration
# - oauth2client: For authentication with Google services
if success:
    success = install_packages("gspread oauth2client",
                             "Google API libraries (gspread, oauth2client)")

# Install interactive widgets for Jupyter/Colab notebooks
# - ipywidgets: For creating interactive controls and dashboards
if success:
    success = install_packages("ipywidgets",
                             "Interactive widgets for Jupyter notebooks")

# Install Ethereum proof verification and analysis libraries
# - eth-utils: For cryptographic functions and general Ethereum utilities
# - rlp: For Recursive Length Prefix encoding used in Ethereum
# - tqdm: For progress visualization in notebook environments
# - graphviz: For visualizing Merkle proofs and contract structures
if success:
    success = install_packages("eth-utils rlp tqdm graphviz",
                             "Ethereum proof verification libraries")

# For Colab environments, install system-level graphviz for visualization

try:
    # Check if running in Google Colab
    import google.colab
    console.print("\n\n🔄 Installing system dependencies for visualization...", style="info")
    # Install graphviz system package (used for rendering graphs)
    subprocess.check_call(['apt-get', '-qq', 'install', 'graphviz'])
    console.print("✓ System-level graphviz installed for advanced visualizations", style="success")

    # Enable enhanced data visualization and export capabilities
    console.print("\n\n🔄 Enabling enhanced data visualization and export...", style="info")

    # Enable interactive data tables from Google Colab
    try:
        from google.colab import data_table
        data_table.enable_dataframe_formatter()
        console.print("✓ Interactive data tables enabled", style="success")
    except ImportError:
        console.print("⚠️ Could not enable interactive data tables", style="warning")

    # Import additional components for file exports and Google Sheets integration
    try:
        # For direct CSV/JSON downloads
        import base64
        import io

        # For Google Sheets export
        from google.colab import output
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaInMemoryUpload

        console.print("✓ Export functionality enabled", style="success")
    except ImportError:
        console.print("⚠️ Some export functions may be limited", style="warning")

    # Verify data table display
    try:
        from IPython.display import display, HTML
        console.print("✓ Data table display verified", style="success")
    except:
        console.print("⚠️ Data table display verification failed", style="warning")

except ImportError:
    console.print("ℹ️ Not running in Colab, skipping system-level installations", style="info")
except Exception as e:
    console.print(f"⚠️ Note: Could not install graphviz system package: {e}. Some visualizations may be limited.", style="warning")

# Final status message with extra spacing
if success:
    console.print("\n\n📦 ✓ All required packages installed successfully!", style="success")
else:
    console.print("\n\n⚠️ [bold]Some packages failed to install. Please check the errors above.[/bold]", style="warning")

# =============================================================================================
# Environment Verification
# =============================================================================================
# Verify all packages imported correctly and set up the analytics environment

try:
    # Core data and utility libraries
    import os
    import json
    import time
    import warnings
    import hashlib

    # Data analysis stack
    import numpy as np
    import pandas as pd

    # Network and graph analysis
    import networkx as nx

    # Visualization libraries
    import matplotlib.pyplot as plt
    import seaborn as sns
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Date handling
    from datetime import datetime, timedelta

    # Collections and data structures
    from collections import defaultdict, Counter

    # Blockchain interaction libraries
    from web3 import Web3
    from web3.exceptions import TransactionNotFound
    from web3.middleware import geth_poa_middleware
    from hexbytes import HexBytes

    # Google Cloud and authentication (for gspread)
    from google.colab import auth # Kept for potential gspread auth if needed in Colab
    from google.oauth2 import service_account
    import gspread
    from oauth2client.client import GoogleCredentials
    from oauth2client.service_account import ServiceAccountCredentials

    # Import ipywidgets components for interactive dashboards
    import ipywidgets as widgets
    from ipywidgets import interact, interactive, fixed, interact_manual

    # Rich components for improved CLI-style output
    from rich.panel import Panel
    from rich.syntax import Syntax
    from rich.table import Table

    # Ethereum specific utilities for cryptography and data structures
    from eth_utils import keccak, to_bytes, to_hex
    import rlp
    from tqdm.notebook import tqdm
    from graphviz import Digraph

    # Suppress warnings for cleaner output
    warnings.filterwarnings('ignore')

    # Configure pandas display settings for better readability
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', 100)      # Reasonable number of rows
    pd.set_option('display.float_format', '{:.6f}'.format)  # Format for amounts

    # Setup Plotly for better Jupyter/Colab integration
    import plotly.io as pio
    pio.templates.default = "plotly_white"

    console.print("\n\n🚀 ✓ Setup complete! Analytics platform initialized.", style="success")
except ImportError as e:
    console.print(f"\n\n❌ Error importing libraries: {e}", style="error")
    console.print("⚠️ Some required packages may not have been installed correctly.", style="warning")

## 🔑 Configuration & Authentication: Connecting to GCP and Ethereum RPC
---

This crucial cell configures PyFlow to connect to Google Cloud Platform services (for authentication and Google Sheets) and the necessary Ethereum networks via GCP's Blockchain Node Engine. **You MUST edit this cell with your specific credentials before running it.**

### 📋 Step 1: Provide Your GCP Credentials

To use GCP's Blockchain RPC and potentially Google Sheets, you need:

1.  **Your GCP Project ID:**
    *   **Purpose:** Identifies your Google Cloud project, primarily used here to scope authentication requests for Google Drive/Sheets access.
    *   **How to Obtain:** If you don't have one, create it at [GCP Console](https://console.cloud.google.com/projectcreate).
    *   **➡️ ACTION REQUIRED:** Find the `GCP_PROJECT_ID` variable in the code below and replace `"YOUR_PROJECT_ID"` with your actual Project ID string.
2.  **Your GCP Blockchain RPC Endpoints (with API Key):**
    *   **Purpose:** Secure URLs to connect to Ethereum Mainnet and testnets via GCP. This is key to interacting with the blockchain using `web3.py`.
    *   **How to Obtain:**
        1.  Go to the [GCP Blockchain Node Engine Console](https://console.cloud.google.com/blockchain/node-engine) in your project.
        2.  Enable the API if you haven't already.
        3.  Copy the **full HTTPS RPC endpoint URL** (including `?key=...`) for **Ethereum Mainnet**. Optionally, copy URLs for testnets (Holesky, Sepolia) if needed.
    *   **➡️ ACTION REQUIRED:** Find the `BLOCKCHAIN_RPC` dictionary in the code below. Replace the placeholder URLs (e.g., `"https://blockchain.googleapis.com/..."`) with your *complete* copied endpoint URLs for `'mainnet'`, `'holesky'`, and `'sepolia'`.

    *Example Format (Use your actual URLs):*
    ```python
    BLOCKCHAIN_RPC = {
        'ethereum': {
            'mainnet': 'https://YOUR_MAINNET_ENDPOINT_URL?key=YOUR_API_KEY',
            # Optional testnets:
            'holesky': 'https://YOUR_HOLESKY_ENDPOINT_URL?key=YOUR_API_KEY',
            'sepolia': 'https://YOUR_SEPOLIA_ENDPOINT_URL?key=YOUR_API_KEY'
        }
    }
    ```

### 🌐 Step 2: Enable Required GCP APIs

Ensure the following APIs are **enabled** in your GCP Project *before* running this cell for authentication and Google Sheets integration to work correctly:

*   **Blockchain Node Engine API:** ([Enable Link](Coming Soon))
*   **Google Drive API:** Required for Google Sheets export/import functionality via `gspread`. ([Enable Link](https://console.cloud.google.com/apis/library/drive.googleapis.com))
*   **Google Sheets API:** Required for Google Sheets export/import functionality via `gspread`. ([Enable Link](https://console.cloud.google.com/apis/library/sheets.googleapis.com))

> #### **💡Tip: Follow README for Video Instructions.**

### 🔐 Step 3: Run the Cell & Authenticate

When you execute the code cell below:

1.  It will define constants (like PYUSD contract addresses) and configurations (trace settings).
2.  It will attempt to **authenticate** your Google account (via a pop-up in Colab) to grant access to the enabled GCP services needed for Google Sheets (Drive, Sheets). Follow the prompts.
3.  It will initialize `web3.py` clients using your provided RPC endpoints.
4.  It will initialize a client for Google Sheets (`gspread`).
5.  It will perform **connection tests** (check RPC node block height) and display a status summary.

> **⚠️ IMPORTANT:** Double-check that you have replaced the placeholder `GCP_PROJECT_ID` and the **full** `BLOCKCHAIN_RPC` URLs before running. The notebook relies heavily on a successful connection to the **Ethereum Mainnet** endpoint via GCP for most subsequent analysis.

In [ ]:
# =============================================================================================
# 📋 Configuration and Authentication for Blockchain Analytics
# =============================================================================================
# This cell configures & Authenticates for blockchain data analysis.

# Import necessary libraries
import google.auth

try:
    from google.colab import auth
except ImportError:
    auth = None
    print("Note: Not running in Google Colab, standard gcloud auth will be used if available.")

import gspread
from web3 import Web3
from web3.middleware import geth_poa_middleware
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress, SpinnerColumn, TextColumn, TimeElapsedColumn
import time
import os

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",
    "success": "spring_green3",
    "warning": "gold3",
    "error": "red3",
    "highlight": "royal_blue1"
})

# Ensure console is created with the theme
console = Console(theme=custom_theme)

# =============================================================================================
# Contract Configuration: PYUSD Stablecoin Addresses
# =============================================================================================

# Main PYUSD Contract addresses (used for querying transactions and events)
PYUSD_PROXY = Web3.to_checksum_address('0x6c3ea9036406852006290770bedfcaba0e23a0e8')
PYUSD_IMPLEMENTATION = Web3.to_checksum_address('0x8EcaE0B0402E29694B3Af35d5943D4631Ee568dC')
SUPPLY_CONTROL_PROXY = Web3.to_checksum_address('0x31d9bDEa6F104606C954f8FE6ba614F1BD347Ec3')
SUPPLY_CONTROL_IMPLEMENTATION = Web3.to_checksum_address('0xFaB5891ED867a1195303251912013b92c4fc3a1D')

# PYUSD Contract Registry with implementation contracts
PYUSD_CONTRACTS = {
    PYUSD_PROXY.lower(): "PYUSD Token",
    PYUSD_IMPLEMENTATION.lower(): "PYUSD Implementation",
    SUPPLY_CONTROL_PROXY.lower(): "Supply Control",
    SUPPLY_CONTROL_IMPLEMENTATION.lower(): "Supply Control Impl"
}

# Define event topics first
TRANSFER_EVENT_TOPIC = Web3.keccak(text="Transfer(address,address,uint256)").hex()
APPROVAL_EVENT_TOPIC = Web3.keccak(text="Approval(address,address,uint256)").hex()
PAUSED_EVENT_TOPIC = Web3.keccak(text="Paused(address)").hex()
UNPAUSED_EVENT_TOPIC = Web3.keccak(text="Unpaused(address)").hex()

# Comprehensive PYUSD configuration
PYUSD_CONFIG = {
    'ethereum': {
        'address': PYUSD_PROXY,
        'implementation': PYUSD_IMPLEMENTATION,
        'decimals': 6,
        'symbol': 'PYUSD',
        'deployment_block': 15921958,
        'transfer_event_topic': TRANSFER_EVENT_TOPIC,
        'approval_event_topic': APPROVAL_EVENT_TOPIC,
        'pause_event_topic': PAUSED_EVENT_TOPIC,
        'unpause_event_topic': UNPAUSED_EVENT_TOPIC
    }
}

PYUSD_ADDRESS_LOWER_ETH = PYUSD_CONFIG['ethereum']['address'].lower()

# PYUSD Function Signature Registry
PYUSD_SIGNATURES = {
    '0xa9059cbb': {"name": "transfer(address,uint256)", "type": "function", "category": "token_movement"},
    '0x095ea7b3': {"name": "approve(address,uint256)", "type": "function", "category": "allowance"},
    '0x23b872dd': {"name": "transferFrom(address,address,uint256)", "type": "function", "category": "token_movement"},
    '0x40c10f19': {"name": "mint(address,uint256)", "type": "function", "category": "supply_change"},
    '0x42966c68': {"name": "burn(uint256)", "type": "function", "category": "supply_change"},
    '0x18160ddd': {"name": "totalSupply()", "type": "function", "category": "view"},
    '0x70a08231': {"name": "balanceOf(address)", "type": "function", "category": "view"},
    '0xdd62ed3e': {"name": "allowance(address,address)", "type": "function", "category": "view"},
    '0x313ce567': {"name": "decimals()", "type": "function", "category": "view"},
    '0x06fdde03': {"name": "name()", "type": "function", "category": "view"},
    '0x95d89b41': {"name": "symbol()", "type": "function", "category": "view"},
    '0x8456cb59': {"name": "pause()", "type": "function", "category": "control"},
    '0x3f4ba83a': {"name": "unpause()", "type": "function", "category": "control"},
    '0x5c975abb': {"name": "paused()", "type": "function", "category": "view"},
    '0xf2fde38b': {"name": "transferOwnership(address)", "type": "function", "category": "admin"},
    '0x8da5cb5b': {"name": "owner()", "type": "function", "category": "view"},
    '0x715018a6': {"name": "renounceOwnership()", "type": "function", "category": "admin"}
}

# PYUSD Event Signature Registry with decoders - using the defined event topics
PYUSD_EVENTS = {
    TRANSFER_EVENT_TOPIC: {
        "name": "Transfer(address,address,uint256)",
        "decoder": lambda topics, data: {
            "from": Web3.to_checksum_address('0x' + topics[1][-40:]),
            "to": Web3.to_checksum_address('0x' + topics[2][-40:]),
            "value": int(data, 16)
        }
    },
    APPROVAL_EVENT_TOPIC: {
        "name": "Approval(address,address,uint256)",
        "decoder": lambda topics, data: {
            "owner": Web3.to_checksum_address('0x' + topics[1][-40:]),
            "spender": Web3.to_checksum_address('0x' + topics[2][-40:]),
            "value": int(data, 16)
        }
    },
    PAUSED_EVENT_TOPIC: {
        "name": "Paused(address)",
        "decoder": lambda topics, data: {
            "account": Web3.to_checksum_address('0x' + topics[1][-40:]) if len(topics) > 1 else None
        }
    },
    UNPAUSED_EVENT_TOPIC: {
        "name": "Unpaused(address)",
        "decoder": lambda topics, data: {
            "account": Web3.to_checksum_address('0x' + topics[1][-40:]) if len(topics) > 1 else None
        }
    }
}

# Tracing configurations
TRACE_CONFIGS = {
    "callTracer": {
        "withLog": True,
        "enableReturnData": True,
        "enableMemory": True,
        "enableStack": True
    },
    "structLog": {
        "disableStorage": False,
        "disableMemory": False,
        "disableStack": False,
        "fullStorage": True
    }
}

# Gas analysis categories
GAS_CATEGORIES = {
    "token_movement": ["transfer", "transferFrom"],
    "supply_change": ["mint", "burn"],
    "allowance": ["approve", "increaseAllowance", "decreaseAllowance"],
    "control": ["pause", "unpause"],
    "admin": ["transferOwnership", "renounceOwnership", "addMinter", "removeMinter"],
    "view": ["balanceOf", "allowance", "totalSupply", "decimals", "name", "symbol", "paused", "owner"],
    "other": []
}


# =============================================================================================
# Data Source Configuration: Google Cloud & Blockchain RPC
# =============================================================================================

# Replace "YOUR_PROJECT_ID" with your actual GCP Project ID (needed for GSheets auth scope)
GCP_PROJECT_ID = "YOUR_PROJECT_ID"
if "YOUR_PROJECT_ID" in GCP_PROJECT_ID or not GCP_PROJECT_ID:
    console.print("[error]🚨 CRITICAL: Please replace placeholder or provide your actual GCP Project ID in GCP_PROJECT_ID.", style="bold red")

# Replace "YOUR_MAINNET_BLOCKCHAIN_RPC_URL", "YOUR_HOLESKY_BLOCKCHAIN_RPC_URL", "YOUR_SEPOLIA_BLOCKCHAIN_RPC_URL" with your FULL RPC endpoint URLs including your API key.
# e.g., 'mainnet': 'https://blockchain.googleapis.com/v1/projects/...........'
BLOCKCHAIN_RPC = {
    'ethereum': {
        'holesky': 'YOUR_HOLESKY_BLOCKCHAIN_RPC_URL',
        'mainnet': 'YOUR_MAINNET_BLOCKCHAIN_RPC_URL',
        'sepolia': 'YOUR_SEPOLIA_BLOCKCHAIN_RPC_URL'
    }
}

# Basic Validation checks for placeholders in URLs
rpc_urls_valid = True
if 'ethereum' not in BLOCKCHAIN_RPC or not BLOCKCHAIN_RPC['ethereum']: # Check if ethereum key exists and has entries
    console.print("[error]🚨 CRITICAL: BLOCKCHAIN_RPC['ethereum'] is missing or empty.", style="bold red")
    rpc_urls_valid = False
else:
    for network, url in BLOCKCHAIN_RPC.get('ethereum', {}).items():
        # More robust check for placeholders or incomplete URLs
        if not url or "xxx" in url.lower() or "/v1/" not in url or "key=" not in url or url.endswith("key="):
            console.print(f"[error]🚨 CRITICAL: RPC URL for '{network}' seems invalid or uses placeholder ('{url}'). Use full URL with API key.", style="bold red")
            rpc_urls_valid = False

# Transaction tracing configuration (for detailed transaction analysis)
# Increased timeout for complex traces
DEFAULT_TRACE_CONFIG = {
    'tracer': 'callTracer',
    'timeout': '120s',
    'tracerConfig': {
        'onlyTopCall': False,
        'withLog': True,
    }
}

STRUCTLOG_TRACE_CONFIG = {
    'tracer': 'structLog',
    # 'timeout': '120s',
}

# =============================================================================================
# Global Client Variables
# =============================================================================================
gc_sheets = None
w3_clients = {}

# =============================================================================================
# Utility Functions: Testing & System Verification
# =============================================================================================

def authenticate_gcp(progress, task_id):
    """Authenticate to GCP, initialize Google Sheets client (updates progress descriptively)."""
    global gc_sheets
    progress.update(task_id, description="[info]Initiating GCP Authentication...")
    if not GCP_PROJECT_ID or "project_id" in GCP_PROJECT_ID:
         progress.update(task_id, description="[error]GCP Auth Failed (No Project ID)")
         return False
    auth_success = False
    gs_init_success = False
    effective_project_id = GCP_PROJECT_ID

    try:
        # Step 1: Authentication (Colab vs. Default)
        creds = None
        if auth: # If google.colab.auth was imported successfully
            progress.update(task_id, description="[info]Waiting for Colab user authentication...")
            auth.authenticate_user(project_id=effective_project_id) # Use project ID here if needed by Colab auth context
            auth_success = True
            progress.update(task_id, description="[info]Colab user authenticated. Getting credentials...")
            creds, _ = google.auth.default() # Get default credentials after Colab auth
        else:
            # Attempt standard ADC (Application Default Credentials) - works in VM, Cloud Shell, local gcloud auth login
            progress.update(task_id, description="[info]Attempting default GCP authentication...")
            try:
                creds, inferred_project_id = google.auth.default()
                if not creds:
                     raise Exception("Could not get default credentials.")
                auth_success = True
                progress.update(task_id, description="[info]Default GCP credentials obtained.")
            except Exception as adc_error:
                progress.update(task_id, description="[error]Default GCP Authentication Failed!")
                console.print(f"❌ Default GCP Auth error: {adc_error}", style="error")
                return False # Hard stop if auth fails

        # Ensure we have credentials before proceeding
        if not creds:
             progress.update(task_id, description="[error]Credentials not obtained after auth attempt.")
             return False

        # Step 2: Initialize Google Sheets Client
        progress.update(task_id, description="[info]Initializing Google Sheets client...")
        gc_sheets = gspread.authorize(creds)
        # Perform a minimal check (e.g., list spreadsheets) if needed, but authorize usually suffices
        # gc_sheets.list_spreadsheet_files(max_results=1)
        gs_init_success = True

        progress.update(task_id, description="[success]GCP Authentication & GSheets Client Initialized")
        return True # Overall success

    except Exception as e:
        error_stage = "GCP Setup Error!"
        if not auth_success:
            error_stage = "GCP Authentication Failed!"
        elif not gs_init_success:
            error_stage = "Google Sheets Client Init Failed!"
        progress.update(task_id, description=f"[error]{error_stage}")
        console.print(f"❌ {error_stage}: {str(e)}", style="error")
        if not gs_init_success: gc_sheets = None
        return False


def initialize_all_web3_clients(progress, task_id):
    """Initializes Web3 clients for all networks (updates progress descriptively)."""
    global w3_clients
    progress.update(task_id, description="[info]Initializing Web3 Clients...")
    w3_clients = {}
    success_count = 0
    total_networks = 0
    if 'ethereum' not in BLOCKCHAIN_RPC or not BLOCKCHAIN_RPC['ethereum']:
        progress.update(task_id, description="[error]No valid RPC Config found!")
        return False
    total_networks = len(BLOCKCHAIN_RPC['ethereum'])
    network_statuses = []

    for network, rpc_url in BLOCKCHAIN_RPC['ethereum'].items():
        progress.update(task_id, description=f"[info]Connecting to {network.capitalize()}...")
        time.sleep(0.1) # Small delay for visual update
        # Find the original url value before the loop modified it
        original_url = BLOCKCHAIN_RPC.get('ethereum', {}).get(network, "")
        is_invalid_url = not original_url or "xxx" in original_url.lower() or "/v1/" not in original_url or "key=" not in original_url or original_url.endswith("key=")
        if is_invalid_url:
            w3_clients[network] = None
            network_statuses.append(f"{network.capitalize()}:[error]Skipped (Invalid URL)[/error]")
            progress.update(task_id, description=f"[warning]Skipping {network.capitalize()} (Invalid URL)...")
            continue
        try:
            # Increased timeout slightly for potentially slower network conditions
            provider = Web3.HTTPProvider(rpc_url, request_kwargs={'timeout': 120})
            w3_client = Web3(provider)
            # Inject middleware only if needed (e.g., for PoA testnets like Goerli, Rinkeby - less relevant for Mainnet/Sepolia/Holesky now)
            # Check chain ID if necessary to decide on middleware, but generally safe to add
            w3_client.middleware_onion.inject(geth_poa_middleware, layer=0)

            # Test connection with get_block_number
            block_num = w3_client.eth.get_block_number()
            w3_clients[network] = w3_client
            success_count += 1
            network_statuses.append(f"{network.capitalize()}:[success]OK (Block: {block_num:,})[/success]")
            progress.update(task_id, description=f"[info]Connecting... ({success_count}/{total_networks} OK)")
        except Exception as e:
            error_short = type(e).__name__
            # Add more detail for common errors
            if "Max retries exceeded" in str(e): error_short = "ConnectionTimeout"
            elif "Failed to establish a new connection" in str(e): error_short = "ConnectionRefused"
            network_statuses.append(f"{network.capitalize()}:[error]{error_short}[/error]")
            w3_clients[network] = None
            progress.update(task_id, description=f"[warning]Failed {network.capitalize()} ({error_short})...")
            console.print(f"[warning]Web3 connection error for {network.capitalize()}: {e}", style="warning")


    final_web3_status = f"[success]Web3 Clients Initialized ({success_count}/{total_networks} OK)"
    if success_count == 0:
        final_web3_status = "[error]Web3 Client Init Failed (All Networks)"
    elif success_count < total_networks:
         final_web3_status = f"[warning]Web3 Clients Initialized ({success_count}/{total_networks} OK)"
    progress.update(task_id, description=final_web3_status)

    # Return True if mainnet client initialized successfully
    return w3_clients.get('mainnet') is not None

# =============================================================================================
# Main Execution: System Initialization and Status Check (with Progress)
# =============================================================================================

# --- Title Display ---
console.print("\n✨ Configuration and Authentication ✨", style="bold cyan3")
console.print("───────────────────────────────────────", style="cyan3")

# Initialize status variables
gcp_auth_success = False
any_web3_success = False
mainnet_ready = False

# Use Rich Progress for initialization steps
# Set transient=True to make the progress bar disappear on completion
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    TimeElapsedColumn(),
    console=console, # Ensure progress uses the themed console
    transient=True # Make spinner disappear when done
) as progress:
    # Add tasks for each step (total=1 means they complete in one update)
    auth_task = progress.add_task("GCP Authentication...", total=1)
    web3_task = progress.add_task("Initializing Web3 Clients...", total=1)

    # --- Run Initialization Steps (functions update progress description) ---
    gcp_auth_success = authenticate_gcp(progress, auth_task)
    progress.update(auth_task, completed=1) # Mark as done

    if rpc_urls_valid:
        any_web3_success = initialize_all_web3_clients(progress, web3_task)
    else:
        progress.update(web3_task, description="[error]Skipped Web3 Init (Invalid URLs)")
    progress.update(web3_task, completed=1)


# --- Get Final Status ---
mainnet_ready = any_web3_success # mainnet_ready directly reflects if mainnet client succeeded
overall_success = gcp_auth_success and mainnet_ready

# --- Display Intermediate Success Messages (Now that progress is done) ---
if gcp_auth_success:
    console.print(f"✓ User authentication successful!", style="success")
    if gc_sheets: console.print(f"✓ Google Sheets client initialized", style="success")
else:
    console.print(f"❌ GCP authentication failed.", style="error")

# Updated Web3 success message based on w3_clients dictionary
if w3_clients:
    connected_nets = [net.capitalize() for net, client in w3_clients.items() if client]
    if connected_nets:
        console.print(f"✓ Web3 clients connected: {', '.join(connected_nets)}", style="success")
    elif rpc_urls_valid: # Only show warning if init was attempted but failed all
         console.print("[warning]Web3 clients initialized, but none connected successfully.", style="warning")
else:
     # This case occurs if rpc_urls_valid was False
     console.print("[error]Web3 client initialization skipped due to invalid RPC URLs.", style="error")


# --- Display Final System Status Summary ---
console.print("\n\n📊 System Status Summary", style="highlight")
# RPC Status
if w3_clients:
    for network, client in w3_clients.items():
        connected = client is not None
        status_msg = "[red]Failed/Skipped[/red]" # Default if not connected
        if connected:
             try:
                 block_num = client.eth.block_number # Get block number again for final status
                 block_num_str = f"(Block #{block_num:,})"
                 status_msg = f"[green]Connected[/green] {block_num_str}"
             except Exception as e:
                  # If client exists but fails here, mark as unresponsive
                  status_msg = f"[orange3]Unresponsive ({type(e).__name__})[/orange3]"
                  w3_clients[network] = None # Clear bad client for consistency
                  console.print(f"[warning]RPC ({network.capitalize()}) became unresponsive: {e}", style="warning")
        else:
            # Add reason if skipped due to invalid URL during init
            original_url = BLOCKCHAIN_RPC.get('ethereum', {}).get(network, "")
            if not original_url or "xxx" in original_url.lower():
                 status_msg = "[red]Skipped (Invalid URL)[/red]"

        console.print(f"  • Ethereum RPC ({network.capitalize()}): {status_msg}")
elif not rpc_urls_valid:
    console.print("  • Ethereum RPC: [red]Skipped (Invalid URLs)[/red]")
else:
    console.print("  • Ethereum RPC: [red]Initialization Failed[/red]")

# GCP Status
status_auth = "[green]Successful[/green]" if gcp_auth_success else "[red]Failed[/red]"
console.print(f"  • GCP Authentication: {status_auth}")

# Google Sheets Status
status_gs = "[green]Initialized[/green]" if gc_sheets else "[red]Not Initialized[/red]"
if not gcp_auth_success: # Also mark skipped if auth failed
    status_gs = "[yellow]Skipped[/yellow]"
console.print(f"  • Google Sheets Client: {status_gs}")

# --- Final Ready/Failure Message with Checkmark ---
if overall_success:
    console.print("\n\n[bold green]✓ Configuration Complete:[/bold green] System Ready for Ethereum Blockchain Analytics (via RPC) and Google Sheets")
    # Optional warnings for testnets can still be useful
    if 'holesky' in w3_clients and w3_clients.get('holesky') is None: console.print("  [warning](Note: Holesky testnet connection failed/skipped/unresponsive)", style="warning")
    if 'sepolia' in w3_clients and w3_clients.get('sepolia') is None: console.print("  [warning](Note: Sepolia testnet connection failed/skipped/unresponsive)", style="warning")
else:
     failure_reasons = []
     if not gcp_auth_success: failure_reasons.append("GCP Auth Failed")
     if not w3_clients.get('mainnet'): failure_reasons.append("Mainnet RPC Connection Failed/Skipped")
     if gcp_auth_success and not gc_sheets: failure_reasons.append("Google Sheets Client Failed") # Check if GSheets failed despite auth success
     if not rpc_urls_valid: failure_reasons.append("Invalid RPC URL Placeholders")


     reason_str = ', '.join(failure_reasons) if failure_reasons else "Unknown Issues"
     console.print(f"\n\n[bold red]❌ Configuration Failed:[/bold red] System setup encountered issues ({reason_str}). Review status messages above.")

## 1.1 🎯 Analysis Targets & Utility Functions
---

This cell handles target validation and prepares the essential utilities needed for transaction/block analysis:

1.  **🌐 Network Verification:**
    *   Verifies connectivity to Ethereum Mainnet and available testnets
    *   Displays current block heights and chain IDs
    *   Attempts reconnection if mainnet client is unavailable

2.  **🎯 Define Analysis Targets:**
    *   Set the specific Ethereum Mainnet **transaction hash** (`TARGET_TX_HASH`) or **block identifier** (`TARGET_BLOCK_IDENTIFIER`) you wish to analyze
    *   **ACTION:** Modify these values to analyze different transactions or blocks relevant to PYUSD
    *   Targets are automatically validated for proper format and existence on-chain

3.  **🔧 Helper Function Library:**
    *   Initializes essential functions used throughout the notebook for:
        *   Making raw RPC requests (`make_rpc_request`) to Ethereum nodes
        *   Decoding PYUSD-specific function calls and events
        *   Formatting blockchain values (ETH/PYUSD amounts, gas)
        *   Handling addresses and transaction data
        *   Creating visualizations for transaction analysis

> **Note:** If a target validation fails, specific diagnostic information will be displayed to help troubleshoot the issue.

In [ ]:
# =============================================================================================
# 🎯 Target Selection & Helper Functions for Tracing
# =============================================================================================
# This cell validates targets, initializes tracing configurations, and sets up helper functions.

# Ensure web3 clients dictionary is loaded from the previous cell
if 'w3_clients' not in locals() or not isinstance(w3_clients, dict):
     raise NameError("Web3 clients dictionary 'w3_clients' not initialized. Please run '🔑 Configuration & Authentication: Connecting to GCP and Ethereum RPC' Cell")

# --- Network Status Verification ---
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    console=console,
    transient=True
) as progress:
    verification_task = progress.add_task("[info]Verifying network connections...", total=1)

    w3_mainnet = w3_clients.get('mainnet')
    if not w3_mainnet or not w3_mainnet.is_connected():
         # Attempt re-initialization with visual feedback
         progress.update(verification_task, description="[warning]⚠️ Mainnet client not found. Attempting re-initialization...")
         web3_task = progress.add_task("[info]Reinitializing Web3 clients...", total=1)
         initialize_all_web3_clients(progress, web3_task)
         progress.update(web3_task, completed=1)

         # Check if reconnection succeeded
         w3_mainnet = w3_clients.get('mainnet')
         if not w3_mainnet or not w3_mainnet.is_connected():
              raise ConnectionError("Cannot proceed without a connected Mainnet Web3 client. Check your RPC endpoints.")

    # Get testnet client if available (for testing function implementations)
    progress.update(verification_task, description="[info]Checking for testnet availability...")
    w3_testnet = w3_clients.get('sepolia') or w3_clients.get('holesky') # Prefer Sepolia if available
    if w3_testnet and w3_testnet.is_connected():
        # Find the network name ('holesky' or 'sepolia')
        testnet_name = next((name for name in ['sepolia', 'holesky'] if w3_clients.get(name) == w3_testnet), None)
    else:
        testnet_name = None # No connections with testnet

    progress.update(verification_task, completed=1)

# --- Network Status Table ---
console.print("[bold cyan3 size=20]🌐 Network Connections[/]", justify="left")
console.print("────────────────────────", style="cyan3")

network_table = Table(show_header=True, header_style="bold cyan3")
network_table.add_column("Network", style="dim")
network_table.add_column("Status", justify="center")
network_table.add_column("Block Height", justify="right")
network_table.add_column("Chain ID", justify="right")

# Add Mainnet info
latest_block_mainnet = w3_mainnet.eth.block_number if w3_mainnet else "N/A"
chain_id_mainnet = w3_mainnet.eth.chain_id if w3_mainnet else "N/A"
network_table.add_row(
    "Ethereum Mainnet",
    "[spring_green3]Connected[/spring_green3]" if w3_mainnet and w3_mainnet.is_connected() else "[red3]Disconnected[/red3]",
    f"{latest_block_mainnet:,}" if isinstance(latest_block_mainnet, int) else str(latest_block_mainnet),
    str(chain_id_mainnet)
)

# Add Testnet info if available
if testnet_name:
    latest_block_testnet = w3_testnet.eth.block_number if w3_testnet else "N/A"
    chain_id_testnet = w3_testnet.eth.chain_id if w3_testnet else "N/A"
    network_table.add_row(
        f"{testnet_name.capitalize()} Testnet",
        "[spring_green3]Connected[/spring_green3]" if w3_testnet and w3_testnet.is_connected() else "[red3]Disconnected[/red3]",
        f"{latest_block_testnet:,}" if isinstance(latest_block_testnet, int) else str(latest_block_testnet),
        str(chain_id_testnet)
    )
else:
    network_table.add_row("Testnet", "[gold3]Not Available[/gold3]", "N/A", "N/A")

# Display the network connection table
console.print(network_table)
console.print("\n\n")  # Empty lines for better spacing

############################################################
# 🎯 DEFINE YOUR ETHEREUM ANALYSIS TARGET
# Set BOTH values below for optimal analysis capabilities.
############################################################

# Transaction Hash for analysis
# Required by certain analysis functions.
TARGET_TX_HASH = "YOUR_TARGET_TX_HASH"

# Block Number/Identifier for analysis
# Required by other analysis functions (often related to block context).
# Use the block containing the target transaction, or the specific block you want to analyze.
TARGET_BLOCK_IDENTIFIER = YOUR_TARGET_BLOCK_NUMBER # Or block hash, "latest", etc.

# Important Notes:
# - Full analysis experience requires both TARGET_TX_HASH and TARGET_BLOCK_IDENTIFIER.
# - Some functions specifically need the block context, others the transaction details.
# - If both values are set, TARGET_TX_HASH takes precedence in situations where
#   only one identifier can be used as the primary target.
############################################################

# Import necessary packages for helper functions
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from graphviz import Digraph
from IPython.display import display, Javascript
from hexbytes import HexBytes
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table
import json
from datetime import datetime

# Plotly configuration for Colab
try:
    display(Javascript('''
        require.config({
            paths: {
                plotly: 'https://cdn.plot.ly/plotly-latest.min.js'
            }
        });
    '''))
except Exception as e:
    console.print(f"[warning]Could not re-configure Plotly: {e}", style="warning")

# --- Target Validation Functions ---
def validate_tx_hash(tx_hash):
    """Validates a transaction hash with detailed diagnostics"""
    if not tx_hash:
        return {
            'valid': False,
            'status': "Missing",
            'status_color': "red3",
            'details': "Transaction hash not provided"
        }

    if not isinstance(tx_hash, str):
        return {
            'valid': False,
            'status': "Invalid Type",
            'status_color': "red3",
            'details': f"Expected string, got {type(tx_hash).__name__}"
        }

    if not tx_hash.startswith('0x'):
        return {
            'valid': False,
            'status': "Invalid Format",
            'status_color': "red3",
            'details': "Transaction hash must start with '0x'"
        }

    if len(tx_hash) < 66:
        return {
            'valid': False,
            'status': "Too Short",
            'status_color': "red3",
            'details': f"Length is {len(tx_hash)}, should be 66 characters"
        }

    if len(tx_hash) > 66:
        return {
            'valid': False,
            'status': "Too Long",
            'status_color': "red3",
            'details': f"Length is {len(tx_hash)}, should be 66 characters (including '0x')"
        }

    # Check if characters are valid hex
    try:
        int(tx_hash[2:], 16)
    except ValueError:
        return {
            'valid': False,
            'status': "Invalid Hex",
            'status_color': "red3",
            'details': "Contains non-hexadecimal characters"
        }

    # At this point, the format is valid, try to retrieve it
    try:
        tx_receipt = w3_mainnet.eth.get_transaction_receipt(tx_hash[:66])  # Truncate to valid length for retrieval
        tx_details = w3_mainnet.eth.get_transaction(tx_hash[:66])

        # Format gas info
        gas_used = tx_receipt.get('gasUsed', 0)
        gas_limit = tx_details.get('gas', 0)
        gas_percentage = (gas_used / gas_limit * 100) if gas_limit else 0

        # Check if transaction was successful
        is_success = tx_receipt.get('status') == 1

        return {
            'valid': True,
            'status': "Found" if is_success else "Failed Transaction",
            'status_color': "spring_green3" if is_success else "gold3",
            'details': f"Block: {tx_receipt.get('blockNumber')}, Gas: {gas_used:,}/{gas_limit:,} ({gas_percentage:.1f}%)",
            'data': {
                'receipt': tx_receipt,
                'transaction': tx_details,
                'success': is_success
            }
        }
    except Exception as e:
        return {
            'valid': True,  # Format is valid but transaction not found
            'status': "Valid Format, Not Found",
            'status_color': "gold3",
            'details': f"Error: {str(e)}"  # No truncation
        }

def validate_block_identifier(block_id):
    """Validates a block identifier with detailed diagnostics"""
    if block_id is None:
        return {
            'valid': False,
            'status': "Missing",
            'status_color': "red3",
            'details': "Block identifier not provided"
        }

    # For integer block numbers
    if isinstance(block_id, int):
        if block_id < 0:
            return {
                'valid': False,
                'status': "Negative Number",
                'status_color': "red3",
                'details': "Block number cannot be negative"
            }

        # Check if block is within realistic range
        latest_block = w3_mainnet.eth.block_number if w3_mainnet else None
        if latest_block and block_id > latest_block:
            return {
                'valid': False,
                'status': "Future Block",
                'status_color': "red3",
                'details': f"Block {block_id:,} exceeds current mainnet height ({latest_block:,})"
            }

        # Block is valid by format, try to retrieve it
        try:
            block_details = w3_mainnet.eth.get_block(block_id)

            # Format timestamp from Unix timestamp
            block_time = datetime.fromtimestamp(block_details.timestamp)

            return {
                'valid': True,
                'status': "Found",
                'status_color': "spring_green3",
                'details': f"Time: {block_time.strftime('%Y-%m-%d %H:%M:%S')}, TX Count: {len(block_details['transactions'])}",
                'data': block_details
            }
        except Exception as e:
            return {
                'valid': True,  # Format is valid but block not found
                'status': "Valid Format, Not Found",
                'status_color': "gold3",
                'details': f"Error: {str(e)}"  # No truncation
            }

    # For hex string or hash
    if isinstance(block_id, str):
        if not block_id.startswith('0x'):
            return {
                'valid': False,
                'status': "Invalid Format",
                'status_color': "red3",
                'details': "Block hash must start with '0x'"
            }

        # Check if characters are valid hex
        try:
            int(block_id[2:], 16)
        except ValueError:
            return {
                'valid': False,
                'status': "Invalid Hex",
                'status_color': "red3",
                'details': "Contains non-hexadecimal characters"
            }

        # Block hash is valid by format, try to retrieve it
        try:
            block_details = w3_mainnet.eth.get_block(block_id)

            # Format timestamp from Unix timestamp
            block_time = datetime.fromtimestamp(block_details.timestamp)

            return {
                'valid': True,
                'status': "Found",
                'status_color': "spring_green3",
                'details': f"Time: {block_time.strftime('%Y-%m-%d %H:%M:%S')}, TX Count: {len(block_details['transactions'])}",
                'data': block_details
            }
        except Exception as e:
            return {
                'valid': True,  # Format is valid but block not found
                'status': "Valid Format, Not Found",
                'status_color': "gold3",
                'details': f"Error: {str(e)}"  # No truncation
            }

    # If not int or str
    return {
        'valid': False,
        'status': "Invalid Type",
        'status_color': "red3",
        'details': f"Expected int or string, got {type(block_id).__name__}"
    }

# --- Target Verification & Information Retrieval with progress indicator ---
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    console=console,
    transient=True
) as progress:
    validate_task = progress.add_task("[info]Validating analysis targets...", total=2)

    # Validate transaction hash
    progress.update(validate_task, description="[info]Analyzing transaction hash...")
    if 'TARGET_TX_HASH' in locals() and TARGET_TX_HASH:
        tx_validation = validate_tx_hash(TARGET_TX_HASH)
        tx_valid = tx_validation['valid']
    else:
        tx_valid = False
        tx_validation = {
            'status': "Skipped",
            'status_color': "gold3",
            'details': "No transaction hash provided"
        }

    progress.update(validate_task, advance=1)

    # Validate block identifier
    progress.update(validate_task, description="[info]Analyzing block identifier...")
    if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
        block_validation = validate_block_identifier(TARGET_BLOCK_IDENTIFIER)
        block_valid = block_validation['valid']
    else:
        block_valid = False
        block_validation = {
            'status': "Skipped",
            'status_color': "gold3",
            'details': "No block identifier provided"
        }

    progress.update(validate_task, completed=1)

# --- Target Status Display (Key-Value Format) ---
console.print("[bold royal_blue1 size=20]🎯 Analysis Targets[/]", justify="left")
console.print("────────────────────", style="royal_blue1")
console.print("\n")  # Extra spacing

# Transaction Target
console.print("[bold]Target Type - Transaction Hash[/bold]")
if 'TARGET_TX_HASH' in locals() and TARGET_TX_HASH:
    console.print(f"[dim]Value:[/dim] [cyan3]{TARGET_TX_HASH}[/cyan3]")
else:
    console.print("[dim]Value:[/dim] Nil")

console.print(f"[dim]Status:[/dim] [{tx_validation['status_color']}]{tx_validation.get('status', 'Unknown')}[/{tx_validation['status_color']}]")
console.print(f"[dim]Diagnostics:[/dim] {tx_validation.get('details', 'No information available')}")

# Separator
console.print("\n" + "─" * 50 + "\n")

# Block Target
console.print("[bold]Target Type - Block[/bold]")
if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    console.print(f"[dim]Value:[/dim] [cyan3]{TARGET_BLOCK_IDENTIFIER}[/cyan3]")
else:
    console.print("[dim]Value:[/dim] Nil")

console.print(f"[dim]Status:[/dim] [{block_validation['status_color']}]{block_validation.get('status', 'Unknown')}[/{block_validation['status_color']}]")
console.print(f"[dim]Diagnostics:[/dim] {block_validation.get('details', 'No information available')}")

console.print("\n")  # Extra spacing

# Show target selection status message
if not tx_valid and not block_valid:
    console.print("[error]⚠️ No valid targets found. Please set either TARGET_TX_HASH or TARGET_BLOCK_IDENTIFIER.[/error]", style="bold red3")
elif tx_valid and block_valid:
    # Check if both targets exist but the transaction validation found the transaction and the block validation found the block
    if tx_validation.get('status') == "Found" and block_validation.get('status') == "Found":
        console.print("[info]ℹ️ Both transaction and block targets are found. Transaction analysis will take priority.[/info]", style="bold royal_blue1")
    else:
        console.print("[info]ℹ️ Both transaction and block targets are provided but at least one wasn't found. Valid target will be used.[/info]", style="bold royal_blue1")
elif tx_valid:
    if tx_validation.get('status') == "Found":
        console.print("[info]✓ Transaction target is valid and found. Ready for analysis.[/info]", style="bold spring_green3")
    else:
        console.print("[warning]⚠️ Transaction target format is valid but transaction wasn't found. Check the hash.[/warning]", style="bold gold3")
elif block_valid:
    if block_validation.get('status') == "Found":
        console.print("[info]✓ Block target is valid and found. Ready for analysis.[/info]", style="bold spring_green3")
    else:
        console.print("[warning]⚠️ Block target format is valid but block wasn't found. Check the number/hash.[/warning]", style="bold gold3")

console.print("\n")  # Empty line for better spacing

# --- Helper Functions ---
def make_rpc_request(method, params, network='mainnet'):
    """Helper function to make raw RPC requests via the specified network's provider."""
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None
    try:
        response = w3_client.provider.make_request(method, params)
        if 'error' in response:
            console.print(f"[error]RPC Error ({method} on {network}): {response['error']['message']} (Code: {response['error']['code']})", style="error")
            return None
        return response.get('result')
    except Exception as e:
        console.print(f"[error]Exception during RPC call ({method} on {network}): {str(e)}", style="error")
        return None

# --- PYUSD Contract Helpers ---
def is_pyusd_contract(address):
    """Checks if an address is a known PYUSD contract."""
    if not address:
        return False
    address_lower = address.lower() if isinstance(address, str) else str(address).lower()
    return address_lower in PYUSD_CONTRACTS

def get_contract_name(address):
    """Gets the friendly name for a contract address."""
    if not address:
        return "Unknown"
    address_lower = address.lower() if isinstance(address, str) else str(address).lower()
    return PYUSD_CONTRACTS.get(address_lower, "Other Contract")

def decode_pyusd_function(input_data):
    """Decodes PYUSD function calls using the signature registry."""
    if not input_data or input_data == '0x':
        return "Empty call data"

    method_sig = input_data[:10]
    if method_sig in PYUSD_SIGNATURES:
        return PYUSD_SIGNATURES[method_sig]["name"]
    return f"Unknown function: {method_sig}"

def get_function_category(input_data):
    """Gets the category of a function from its input data."""
    if not input_data or input_data == '0x':
        return "other"

    method_sig = input_data[:10]
    if method_sig in PYUSD_SIGNATURES:
        return PYUSD_SIGNATURES[method_sig]["category"]
    return "other"

def decode_pyusd_event(topic0, topics, data):
    """Decodes PYUSD events using the event registry."""
    if not topic0 or topic0 not in PYUSD_EVENTS:
        return {"name": "Unknown event", "details": "Cannot decode"}

    event_info = PYUSD_EVENTS[topic0]

    try:
        decoded = event_info["decoder"](topics, data)
        return {
            "name": event_info["name"],
            "decoded": decoded
        }
    except Exception as e:
        return {
            "name": event_info["name"],
            "error": str(e)
        }

# --- Formatting Helpers ---
def format_value_pyusd(value_raw):
    """Formats raw PYUSD value (int) to decimal string."""
    if value_raw is None: return "0 PYUSD"
    try:
        decimals = PYUSD_CONFIG['ethereum']['decimals']
        value_float = int(value_raw) / (10**decimals)
        return f"{value_float:,.{decimals}f} PYUSD"
    except (ValueError, TypeError):
        return "Invalid PYUSD Value"

def format_value_eth(value_wei):
    """Formats Wei value to Ether string."""
    if value_wei is None: return "0 ETH"
    try:
        # Ensure value_wei is int or can be converted (handle hex strings from traces)
        if isinstance(value_wei, str):
            value_int = int(value_wei, 16)
        else:
             value_int = int(value_wei)
        # Use mainnet client for formatting ETH value regardless of target network
        if w3_mainnet: # Check if mainnet client exists
            return f"{w3_mainnet.from_wei(value_int, 'ether'):.6f} ETH"
        else:
            return f"{value_int / 1e18:.6f} ETH (approx)" # Fallback if mainnet client missing
    except (ValueError, TypeError, AttributeError):
        return "Invalid ETH Value"

def format_gas(gas):
    """Formats gas value (int or hex string)."""
    if gas is None: return "N/A"
    try:
       gas_int = int(gas, 16) if isinstance(gas, str) and gas.startswith('0x') else int(gas)
       return f"{gas_int:,}"
    except (ValueError, TypeError):
        return "Invalid Gas Value"

def shorten_address(address):
    """Shortens an Ethereum address for display."""
    if not isinstance(address, str) or not address.startswith('0x') or len(address) != 42:
        return str(address) # Return original if not a valid address string
    return f"{address[:6]}...{address[-4:]}"

def display_json(data, title="JSON Output"):
    """Pretty prints JSON data using Rich."""
    if data is None:
        console.print(f"[warning]{title}: No data to display.", style="warning")
        return
    try:
        # Use default=str to handle potential non-serializable types like HexBytes
        json_str = json.dumps(data, indent=2, default=str)
        console.print(Panel(Syntax(json_str, "json", theme="default", line_numbers=False),
                      title=title, border_style="cyan3", expand=False))
    except Exception as e:
        console.print(f"[error]Could not display JSON for {title}: {e}", style="error")
        # Fallback to printing raw data (limited length)
        try:
            raw_str = str(data)
            console.print(raw_str[:1000] + ("..." if len(raw_str) > 1000 else ""))
        except Exception:
            console.print("[error]Could not print raw data fallback.", style="error")

# --- Visualization Helpers ---
def create_gas_usage_chart(gas_data, title="Gas Usage Distribution"):
    """Creates a pie chart showing gas usage distribution."""
    try:
        fig = px.pie(gas_data, values='gas_used', names='category',
                      title=title)
        fig.update_layout(template="plotly_white")
        return fig
    except Exception as e:
        console.print(f"[warning]Could not create gas usage chart: {e}", style="warning")
        return None

def create_call_sequence_chart(calls_df, highlight_pyusd=True):
    """Creates a bar chart showing the sequence of calls with gas usage."""
    try:
        if highlight_pyusd:
            fig = px.bar(calls_df, x='id', y='gasUsed', color='is_pyusd',
                         title='Gas Usage by Call Sequence',
                         hover_data=['type', 'depth', 'from', 'to'])
        else:
            fig = px.bar(calls_df, x='id', y='gasUsed',
                         title='Gas Usage by Call Sequence',
                         hover_data=['type', 'depth', 'from', 'to'])

        fig.update_layout(template="plotly_white")
        return fig
    except Exception as e:
        console.print(f"[warning]Could not create call sequence chart: {e}", style="warning")
        return None

# --- Final Tracing Configuration Setup ---
# Set up trace configuration based on targets
if tx_valid and tx_validation.get('status') == "Found":
    # If valid transaction hash found, prepare for transaction tracing
    ACTIVE_TRACE_CONFIG = DEFAULT_TRACE_CONFIG.copy()
    console.print(f"[info]Trace configuration ready for transaction analysis", style="info")
elif block_valid and block_validation.get('status') == "Found":
    # If valid block found, prepare for block tracing
    ACTIVE_TRACE_CONFIG = STRUCTLOG_TRACE_CONFIG.copy()
    console.print(f"[info]Trace configuration ready for block analysis", style="info")
else:
    # If no valid targets, set a default trace config
    ACTIVE_TRACE_CONFIG = DEFAULT_TRACE_CONFIG.copy()
    console.print(f"[warning]Using default trace configuration (no valid target found)", style="warning")

# Final status message
status_icon = "✓" if (tx_valid and tx_validation.get('status') == "Found") or (block_valid and block_validation.get('status') == "Found") else "⚠️"
status_style = "bold spring_green3" if status_icon == "✓" else "bold gold3"
console.print(f"[{status_style}]{status_icon} Target selection and helper functions initialized[/{status_style}]")

## 1.2 🔍 `debug_traceTransaction` - Deep Dive into Transaction Execution
---
This section utilizes the powerful `debug_traceTransaction` RPC method to dissect the internal workings of a specific PYUSD transaction defined by `TARGET_TX_HASH`. This goes far beyond standard block explorers by revealing the step-by-step execution flow within the EVM. We will explore two main tracers: `callTracer` and `structLog`.

### 1.2.1 Using `callTracer`: Mapping Internal Calls, Gas & Events (Recommended)

The `callTracer` provides a structured, hierarchical view of the transaction's execution flow. It's generally the most useful tracer for understanding high-level interactions, gas consumption patterns, and event emissions.

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **Method:** `debug_traceTransaction` with `tracer: "callTracer"`
> *   **Multiplier:** `50x` (Consumes 50x the quota/cost of a basic call)
> *   **GCP Advantage:** Performing this detailed trace is computationally intensive. GCP's generous free quotas for this high-multiplier method make such in-depth analysis accessible and cost-effective.
> *   **PYUSD Insight:** `callTracer` allows us to:
>     *   Visualize the **exact interaction path** when PYUSD interacts with other DeFi protocols (e.g., DEXs, lending platforms).
>     *   Identify specific **internal PYUSD function calls** (`transfer`, `approve`, `mint`, `burn`) and their parameters within the overall transaction.
>     *   Pinpoint **gas consumption** within specific PYUSD operations vs. external contract interactions.
>     *   Verify the **emission and content** of crucial PYUSD events like `Transfer` and `Approval`.

**Analysis Workflow:**

1.  **Fetch Trace:** The code calls `debug_traceTransaction` using the `TARGET_TX_HASH` and the `callTracer` configuration.
2.  **Parse Data:** The complex JSON response is processed by `parse_call_trace` to extract structured information about calls, logs, gas, and state changes.
3.  **Visualize & Summarize:** The results are presented through:
    *   **Trace Overview & Metrics:** Key stats like call count, depth, gas, and status.
    *   **Interaction Graphs:** High-level contract interactions and detailed call sequences (Interactive Plotly). PYUSD calls are highlighted.
    *   **Token Flow:** A specific graph visualizing PYUSD movements (transfers, mints, burns).
    *   **State & Gas Analysis:** Tables and charts showing PYUSD state changes and gas usage breakdown.
    *   **Event Log Analysis:** Decoded PYUSD events emitted.
    *   **Data Tables:** DataFrames showing key calls and high gas usage operations.
    *   **Recommendations:** Automated observations based on trace patterns.
    *   **Export Options:** Download parsed data (CSV/JSON) or export a report to Google Sheets (Colab).

**💡 What to Look For:**
*   **Call Graph:** Observe the sequence and depth of calls. Identify the green/blue highlighted nodes representing PYUSD/Supply Controller interactions. Note the gas usage (`Gas: ...`) on each node.
*   **Token Flow Graph:** Track how PYUSD moved between addresses.
*   **Gas Usage Pie Chart:** See which *types* of operations (transfer, approval, supply change) consumed the most gas.
*   **Event Table:** Correlate events like `Transfer` with the calls shown in the graph.
*   **Recommendations:** Check for automated insights about gas or complexity.

In [ ]:
# =============================================================================================
# 🔬 Trace Transaction using debug_traceTransaction (callTracer)
# =============================================================================================
# This cell validates transaction targets, initializes blockchain tracing configurations, and sets up helper functions.
# It prepares the environment for detailed transaction analysis by:
# - Validating the transaction hash format and existence
# - Configuring tracer parameters for debug_traceTransaction
# - Setting up utility functions for address formatting, value conversion, and data processing
# - Initializing transaction-specific constants and analysis options

import base64
import json
import os
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import Javascript
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from rich.console import Console
from rich.table import Table
from rich.panel import Panel

def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, tx_hash):
    """Export analysis data to Google Sheets with rich formatting and visualization references."""
    # Show loading message
    console.print("[cyan3]Exporting to Google Sheets...", style="info")

    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"PYUSD TX Analysis {tx_hash[:10]} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Transaction Analysis")

        # Set up a header with transaction info
        header_values = [
            ["PYUSD Transaction Analysis"],
            [f"Transaction: {tx_hash}"],
            [f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header with bold text and colored background
        worksheet.format("A1:A1", {
            "textFormat": {"bold": True, "fontSize": 14},
            "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}
        })

        worksheet.format("A2:A3", {
            "textFormat": {"bold": True, "fontSize": 12}
        })

        current_row = 5  # Start after header

        # 1. Add transaction stats summary with improved formatting
        if "transaction_stats" in data_dict:
            stats = data_dict["transaction_stats"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Transaction Metrics"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add stats data
            stats_rows = []
            stats_rows.append(["Metric", "Value"])  # Header row
            for key, value in stats.items():
                # Format keys and values appropriately
                formatted_key = key.replace("_", " ").title()
                if key == 'total_gas':
                    formatted_value = f"{value:,} gas units"
                else:
                    formatted_value = str(value)
                stats_rows.append([formatted_key, formatted_value])

            # Add stats table
            stats_start_row = current_row
            worksheet.update(f"A{stats_start_row}", stats_rows)

            # Format stats table header
            worksheet.format(f"A{stats_start_row}:B{stats_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(stats_rows) + 1  # Add space after table

        # 2. Add Contract Interaction Graph reference
        worksheet.update(f"A{current_row}", [["Contract Interaction Overview"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Contract interaction visualization is available in the notebook"]])
        current_row += 2

        # 3. Add Call Graph Visualization reference
        worksheet.update(f"A{current_row}", [["Detailed Call Graph Visualization"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 0.7, "green": 0.8, "blue": 1.0}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Detailed call graph visualization is available in the notebook"]])
        current_row += 2

        # 4. Add gas usage section
        if "gas_distribution" in data_dict and data_dict["gas_distribution"]:
            gas_data = data_dict["gas_distribution"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Gas Usage Analysis"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 1.0, "green": 0.9, "blue": 0.7}
            })
            current_row += 1

            # Create gas usage table
            gas_header = ["Category", "Gas Used", "Percentage"]
            gas_rows = [gas_header]

            total_gas = sum(item["gas_used"] for item in gas_data)
            for item in gas_data:
                category = item["category"].replace("_", " ").title()
                gas_used = item["gas_used"]
                percentage = (gas_used / total_gas * 100) if total_gas > 0 else 0
                gas_rows.append([category, f"{gas_used:,}", f"{percentage:.1f}%"])

            gas_table_row = current_row
            worksheet.update(f"A{gas_table_row}", gas_rows)

            # Format gas table headers
            worksheet.format(f"A{gas_table_row}:C{gas_table_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(gas_rows) + 1

            # Add pie chart reference
            worksheet.update(f"A{current_row}", [["📊 Gas usage pie chart visualization is available in the notebook"]])
            current_row += 2

        # 5. Add PYUSD Token Flow section
        if "pyusd_transfers" in data_dict and data_dict["pyusd_transfers"]:
            transfers = data_dict["pyusd_transfers"]

            # Add section title
            worksheet.update(f"A{current_row}", [["PYUSD Token Flow Analysis"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.7, "green": 1.0, "blue": 0.8}
            })
            current_row += 1

            # Add description
            worksheet.update(f"A{current_row}", [["This shows the movement of PYUSD tokens in this transaction."]])
            current_row += 1

            # Add transfer data as a table
            transfer_header = ["From", "To", "Amount", "Gas Used"]
            transfer_rows = [transfer_header]

            for transfer in transfers:
                from_addr = shorten_address(transfer["from"])
                to_addr = shorten_address(transfer["to"])
                amount = format_value_pyusd(transfer["amount"])
                gas = f"{transfer.get('gas_used', 0):,}"
                transfer_rows.append([from_addr, to_addr, amount, gas])

            transfer_table_row = current_row
            worksheet.update(f"A{transfer_table_row}", transfer_rows)

            # Format transfer table headers
            worksheet.format(f"A{transfer_table_row}:D{transfer_table_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(transfer_rows) + 1

            # Add flow graph reference
            worksheet.update(f"A{current_row}", [["🔄 Token flow visualization is available in the notebook"]])
            current_row += 2

        # 6. Add PYUSD State Changes
        if "state_changes" in data_dict and data_dict["state_changes"]:
            state_changes = data_dict["state_changes"]

            # Add section title
            worksheet.update(f"A{current_row}", [["PYUSD State Changes"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 1.0, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # Add description
            worksheet.update(f"A{current_row}", [["The following state changes occurred in PYUSD contracts:"]])
            current_row += 1

            # Create state changes table
            if isinstance(state_changes, list) and state_changes:
                # Define headers
                headers = ["contract", "function", "type", "amount", "gas_used"]
                display_headers = ["Contract", "Function", "Type", "Amount", "Gas Used"]

                # Create the table data
                state_rows = [display_headers]  # Header row
                for change in state_changes:
                    row_data = []
                    for key in headers:
                        if key == "amount" and "amount" in change:
                            # Format PYUSD amounts nicely
                            value = format_value_pyusd(change["amount"])
                        elif key == "gas_used":
                            value = f"{change.get(key, 0):,}"
                        else:
                            value = str(change.get(key, ""))
                        row_data.append(value)
                    state_rows.append(row_data)

                # Add to sheet
                state_start_row = current_row
                worksheet.update(f"A{state_start_row}", state_rows)

                # Format headers
                header_range = f"A{state_start_row}:E{state_start_row}"
                worksheet.format(header_range, {
                    "textFormat": {"bold": True},
                    "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                })

                current_row += len(state_rows) + 1

        # 7. Add Event Analysis summary
        if "logs_df" in data_dict and isinstance(data_dict["logs_df"], pd.DataFrame) and not data_dict["logs_df"].empty:
            logs_df = data_dict["logs_df"]
            pyusd_logs = logs_df[logs_df['is_pyusd']] if 'is_pyusd' in logs_df.columns else pd.DataFrame()

            if not pyusd_logs.empty:
                # Add section title
                worksheet.update(f"A{current_row}", [["PYUSD Events Analysis"]])
                worksheet.format(f"A{current_row}:A{current_row}", {
                    "textFormat": {"bold": True, "fontSize": 12},
                    "backgroundColor": {"red": 0.7, "green": 1.0, "blue": 0.8}
                })
                current_row += 1

                # Add summary count
                worksheet.update(f"A{current_row}", [[f"Found {len(pyusd_logs)} PYUSD events in this transaction."]])
                current_row += 1

                # Add event counts if available
                if 'event_name' in pyusd_logs.columns:
                    event_counts = pyusd_logs['event_name'].value_counts()

                    # Create event counts table
                    event_table = [["Event Type", "Count"]]  # Header row
                    for event, count in event_counts.items():
                        event_table.append([event, str(count)])

                    # Add to sheet
                    event_table_row = current_row
                    worksheet.update(f"A{event_table_row}", event_table)

                    # Format headers
                    worksheet.format(f"A{event_table_row}:B{event_table_row}", {
                        "textFormat": {"bold": True},
                        "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                    })

                    current_row += len(event_table) + 1

                # Add event details
                worksheet.update(f"A{current_row}", [["PYUSD Event Details:"]])
                worksheet.format(f"A{current_row}:A{current_row}", {"textFormat": {"bold": True}})
                current_row += 1

                # Display key event details
                event_cols = ['contract', 'event_name', 'details']
                event_cols = [col for col in event_cols if col in pyusd_logs.columns]

                if event_cols:
                    # Convert to list for sheet
                    event_data = [event_cols]  # Header row
                    for _, row in pyusd_logs[event_cols].iterrows():
                        # Format each value appropriately
                        row_values = []
                        for col in event_cols:
                            if pd.isnull(row[col]):
                                value = ""
                            else:
                                value = str(row[col])
                            row_values.append(value)
                        event_data.append(row_values)

                    # Add to sheet
                    event_start_row = current_row
                    worksheet.update(f"A{event_start_row}", event_data)

                    # Format headers
                    header_range = f"A{event_start_row}:{chr(65+len(event_cols)-1)}{event_start_row}"
                    worksheet.format(header_range, {
                        "textFormat": {"bold": True},
                        "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                    })

                    current_row += len(event_data) + 1

                # Add transfer value summary if available
                if 'amount' in pyusd_logs.columns and 'is_transfer' in pyusd_logs.columns:
                    transfer_logs = pyusd_logs[pyusd_logs['is_transfer']]
                    if not transfer_logs.empty:
                        total_transferred = transfer_logs['amount'].sum()
                        worksheet.update(f"A{current_row}", [[f"Total PYUSD transferred: {format_value_pyusd(total_transferred)}"]])
                        current_row += 2

        # 8. Add Recommendations
        if "recommendations" in data_dict and data_dict["recommendations"]:
            recommendations = data_dict["recommendations"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Analysis Observations & Recommendations"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.7, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add each recommendation
            for rec in recommendations:
                worksheet.update(f"A{current_row}", [[rec]])
                current_row += 1

            current_row += 1  # Extra space

        # 9. Add main DataFrame data (selected columns only for better readability)
        if not df.empty:
            # Add a section title
            worksheet.update(f"A{current_row}", [["Key Contract Calls"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # Select important columns to display
            display_cols = ['id', 'type', 'depth', 'contract', 'function_category', 'gasUsed', 'is_pyusd']
            display_cols = [col for col in display_cols if col in df.columns]

            # First show PYUSD calls
            pyusd_calls = df[df['is_pyusd']] if 'is_pyusd' in df.columns else pd.DataFrame()

            if not pyusd_calls.empty:
                worksheet.update(f"A{current_row}", [[f"Found {len(pyusd_calls)} PYUSD-related calls:"]])
                current_row += 1

                # Convert DataFrame to list of lists for the worksheet
                pyusd_df_values = [display_cols] + pyusd_calls[display_cols].values.tolist()

                # Format values for display
                for i in range(1, len(pyusd_df_values)):
                    for j, col in enumerate(display_cols):
                        val = pyusd_df_values[i][j]
                        if col == 'gasUsed' and pd.notnull(val):
                            pyusd_df_values[i][j] = f"{val:,}"
                        elif pd.isnull(val):
                            pyusd_df_values[i][j] = "NULL"
                        else:
                            pyusd_df_values[i][j] = str(val)

                worksheet.update(f"A{current_row}", pyusd_df_values)

                # Format the DataFrame header
                worksheet.format(f"A{current_row}:{chr(65+len(display_cols)-1)}{current_row}", {
                    "textFormat": {"bold": True},
                    "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                })

                # Add alternating row colors for readability
                data_rows = len(pyusd_df_values)
                for i in range(2, data_rows + 1, 2):
                    row_num = current_row + i - 1
                    worksheet.format(f"A{row_num}:{chr(65+len(display_cols)-1)}{row_num}", {
                        "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                    })

                current_row += len(pyusd_df_values) + 2

            # Then show highest gas usage calls
            worksheet.update(f"A{current_row}", [["Highest Gas Usage Calls:"]])
            worksheet.format(f"A{current_row}:A{current_row}", {"textFormat": {"bold": True}})
            current_row += 1

            # Get top 5 by gas usage
            high_gas_cols = ['id', 'type', 'contract', 'function_category', 'gasUsed']
            high_gas_cols = [col for col in high_gas_cols if col in df.columns]

            high_gas_calls = df.nlargest(5, 'gasUsed') if 'gasUsed' in df.columns else pd.DataFrame()

            if not high_gas_calls.empty:
                # Convert to list for sheet
                high_gas_data = [high_gas_cols]  # Header row
                for _, row in high_gas_calls[high_gas_cols].iterrows():
                    # Format each value appropriately
                    row_values = []
                    for col in high_gas_cols:
                        if col == 'gasUsed':
                            value = f"{row[col]:,}" if pd.notnull(row[col]) else "0"
                        elif pd.isnull(row[col]):
                            value = "NULL"
                        else:
                            value = str(row[col])
                        row_values.append(value)
                    high_gas_data.append(row_values)

                # Add to sheet
                worksheet.update(f"A{current_row}", high_gas_data)

                # Format headers
                header_range = f"A{current_row}:{chr(65+len(high_gas_cols)-1)}{current_row}"
                worksheet.format(header_range, {
                    "textFormat": {"bold": True},
                    "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
                })

                # Add alternating row colors
                for i in range(2, len(high_gas_data) + 1, 2):
                    row_num = current_row + i - 1
                    worksheet.format(f"A{row_num}:{chr(65+len(high_gas_cols)-1)}{row_num}", {
                        "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                    })

        # Try to auto-resize columns for better readability
        try:
            worksheet.columns_auto_resize(0, 10)  # Attempt to resize first 10 columns
        except:
            pass  # Ignore if not supported

        # Return spreadsheet URL and title for opening
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        return (spreadsheet_url, sheet_title)

    except Exception as e:
        console.print(f"[error]Error creating Google Sheet: {str(e)}", style="error")
        raise Exception(f"Error creating Google Sheet: {str(e)}")

# Function to display loading indicator while rendering visualization
def show_loading_indicator():
    """Display a loading indicator while rendering graphs"""
    loading_html = """
    <div style="display: flex; justify-content: center; align-items: center; height: 50px;">
        <div style="text-align: center;">
            <div class="spinner-border" role="status">
                <span class="sr-only">Loading...</span>
            </div>
            <p style="margin-top: 10px; color: #555;">Generating visualization...</p>
        </div>
    </div>
    <style>
    .spinner-border {
        display: inline-block;
        width: 2rem;
        height: 2rem;
        vertical-align: text-bottom;
        border: 0.25em solid currentColor;
        border-right-color: transparent;
        border-radius: 50%;
        animation: spinner-border .75s linear infinite;
    }
    @keyframes spinner-border {
        to { transform: rotate(360deg); }
    }
    </style>
    """
    display(HTML(loading_html))

# Function to get human-readable function descriptions
def get_function_description(input_data, is_pyusd, contract_name):
    """Get a human-readable function description."""
    if not input_data or input_data == '0x':
        return "Contract Creation" if not input_data else "ETH Transfer"

    # Extract method signature (first 10 characters including 0x)
    method_sig = input_data[:10] if len(input_data) >= 10 else input_data

    # Check PYUSD signatures first
    if method_sig in PYUSD_SIGNATURES:
        function_info = PYUSD_SIGNATURES[method_sig]
        return function_info["name"]

    # Check common ERC20/generic function signatures
    common_sigs = {
        '0xa9059cbb': "transfer(address,uint256)",
        '0x095ea7b3': "approve(address,uint256)",
        '0x23b872dd': "transferFrom(address,address,uint256)",
        '0x18160ddd': "totalSupply()",
        '0x70a08231': "balanceOf(address)",
        '0xdd62ed3e': "allowance(address,address)",
        '0x06fdde03': "name()",
        '0x95d89b41': "symbol()",
        '0x313ce567': "decimals()",
        '0x8da5cb5b': "owner()",
        '0x715018a6': "renounceOwnership()",
        '0xf2fde38b': "transferOwnership(address)",
        '0x01ffc9a7': "supportsInterface(bytes4)",
        '0x3644e515': "DOMAIN_SEPARATOR()",
        '0x7ecebe00': "nonces(address)",
        '0xd505accf': "permit(address,address,uint256,uint256,uint8,bytes32,bytes32)"
    }

    return common_sigs.get(method_sig, f"Function {method_sig}")

# function for creating interactive Plotly Contract Interaction Graph
def create_plotly_contract_interaction_graph(contract_interactions):
    """Creates an interactive Plotly Network graph for contract interactions with directional arrows"""
    import math

    if not contract_interactions:
        return None

    # Create a networkx graph from the interaction data
    G = nx.DiGraph()

    # Add nodes for all contracts in interactions
    contracts_seen = set()
    for src, dst in contract_interactions:
        if src not in contracts_seen:
            src_name = PYUSD_CONTRACTS.get(src, "External Contract")
            G.add_node(src, name=src_name, is_pyusd=(src in PYUSD_CONTRACTS))
            contracts_seen.add(src)

        if dst not in contracts_seen:
            dst_name = PYUSD_CONTRACTS.get(dst, "External Contract")
            G.add_node(dst, name=dst_name, is_pyusd=(dst in PYUSD_CONTRACTS))
            contracts_seen.add(dst)

        # Add edge
        G.add_edge(src, dst)

    # Calculate layout with more spacing for readability
    pos = nx.spring_layout(G, seed=42, k=1.5)

    # Create edge traces with arrows
    edge_traces = []

    for edge in G.edges():
        src, dst = edge
        src_name = G.nodes[src]['name']
        dst_name = G.nodes[dst]['name']

        x0, y0 = pos[src]
        x1, y1 = pos[dst]

        # Calculate direction vector for arrow
        dx = x1 - x0
        dy = y1 - y0

        # Normalize the vector
        length = math.sqrt(dx**2 + dy**2)
        if length > 0:
            udx = dx / length
            udy = dy / length
        else:
            udx, udy = 0, 0

        # Position arrow slightly before the destination node (80% along the edge)
        arrow_ratio = 0.8
        arrow_x = x0 + arrow_ratio * dx
        arrow_y = y0 + arrow_ratio * dy

        # Angle for the arrow in degrees
        angle = math.degrees(math.atan2(dy, dx))

        # Main edge line
        edge_trace = go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            line=dict(width=1.5, color='rgba(50, 50, 50, 0.8)'),
            hoverinfo='text',
            text=f"From: {src_name}<br>To: {dst_name}<br>From address: {src}<br>To address: {dst}",
            mode='lines',
            showlegend=False
        )

        # Arrow marker
        arrow_trace = go.Scatter(
            x=[arrow_x],
            y=[arrow_y],
            mode='markers',
            marker=dict(
                symbol='triangle-right',
                size=12,
                color='rgba(50, 50, 50, 0.8)',
                angle=angle  # Apply the calculated angle
            ),
            hoverinfo='none',
            showlegend=False
        )

        edge_traces.append(edge_trace)
        edge_traces.append(arrow_trace)

    # Create node trace
    node_x = []
    node_y = []
    node_colors = []
    node_sizes = []
    hover_texts = []
    node_addresses = []

    for node in G.nodes():
        node_data = G.nodes[node]
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Node color based on contract type
        if node_data['is_pyusd']:
            if "PYUSD Token" in node_data['name']:
                node_colors.append('rgba(144, 238, 144, 0.9)')  # palegreen
            elif "Supply Control" in node_data['name']:
                node_colors.append('rgba(135, 206, 250, 0.9)')  # lightskyblue
            else:
                node_colors.append('rgba(224, 255, 255, 0.9)')  # lightcyan
        else:
            node_colors.append('rgba(211, 211, 211, 0.9)')  # lightgray

        # Node size: bigger for PYUSD contracts
        node_sizes.append(25 if node_data['is_pyusd'] else 18)

        # Full address for node label
        node_addresses.append(node)

        # Hover text with full contract information
        hover_texts.append(f"<b>{node_data['name']}</b><br>Address: {node}")

    # Create node trace with text labels showing full addresses
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_addresses,
        textposition="bottom center",
        hovertext=hover_texts,
        marker=dict(
            showscale=False,
            color=node_colors,
            size=node_sizes,
            line=dict(width=1, color='#000')
        ),
        textfont=dict(
            family="monospace",
            size=10,
            color="black"
        )
    )

    # Add legend traces for different node types
    legend_traces = [
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(144, 238, 144, 0.9)'),
            name='PYUSD Token',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(135, 206, 250, 0.9)'),
            name='Supply Control',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(211, 211, 211, 0.9)'),
            name='External Contract',
            showlegend=True
        )
    ]

    # Create figure with styled layout
    fig = go.Figure()

    # Add all traces
    for trace in edge_traces:
        fig.add_trace(trace)

    fig.add_trace(node_trace)

    for trace in legend_traces:
        fig.add_trace(trace)

    # Style the figure
    fig.update_layout(
        title='<b>Contract Interaction Overview</b>',
        titlefont=dict(size=16),
        showlegend=True,
        legend=dict(
            title="Contract Types",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(255, 255, 255, 0.8)"
        ),
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white",
        height=600,
        paper_bgcolor='rgba(255,255,255,0.8)',
        plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig

# function for creating interactive Plotly Call Graph
def create_plotly_call_graph(call_data_list):
    """Creates an interactive Plotly Network graph for the call trace hierarchy"""
    if not call_data_list:
        return None

    # Process relationships and parent-child connections
    # We need to reconstruct parent/child relationships based on depth
    for i, call in enumerate(call_data_list):
        # Initialize parent_id field if it doesn't exist
        if 'parent_id' not in call:
            call['parent_id'] = None

        # If not root node, find parent
        if call['depth'] > 0 and i > 0:
            # Look backward for potential parents at the previous depth level
            for j in range(i-1, -1, -1):
                potential_parent = call_data_list[j]
                if potential_parent['depth'] == call['depth'] - 1:
                    call['parent_id'] = potential_parent['id']
                    break

    # Create a networkx graph
    G = nx.DiGraph()

    # Add all nodes with their attributes
    for call in call_data_list:
        G.add_node(call['id'], **call)

    # Add edges based on parent-child relationships
    for call in call_data_list:
        if call['parent_id']:
            G.add_edge(call['parent_id'], call['id'])

    # Try using graphviz layout if available
    try:
        import pygraphviz as pgv
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')  # Hierarchical layout
    except:
        # Fallback to custom hierarchical layout
        pos = {}
        depth_to_nodes = {}

        for node, data in G.nodes(data=True):
            depth = data['depth']
            if depth not in depth_to_nodes:
                depth_to_nodes[depth] = []
            depth_to_nodes[depth].append(node)

        # Calculate positions based on depth
        max_depth = max(depth_to_nodes.keys()) if depth_to_nodes else 0

        for depth, nodes in depth_to_nodes.items():
            nodes.sort()  # For consistent layout
            node_count = len(nodes)
            for i, node in enumerate(nodes):
                # Horizontal spacing based on node count, vertical based on depth
                x_pos = i / (node_count + 1) if node_count > 1 else 0.5
                y_pos = 1.0 - (depth / (max_depth + 1)) if max_depth > 0 else 0.5
                pos[node] = (x_pos, y_pos)

    # Create edge traces with different colors by call type
    edge_traces_by_type = {}
    edge_types = set()

    for edge in G.edges():
        source, target = edge
        source_data = G.nodes[source]
        target_data = G.nodes[target]

        call_type = target_data['type']
        edge_types.add(call_type)

        if call_type not in edge_traces_by_type:
            edge_traces_by_type[call_type] = {
                'x': [], 'y': [], 'text': [], 'color': '', 'style': '', 'width': 1
            }

            # Set color and style based on call type
            if call_type == 'DELEGATECALL':
                edge_traces_by_type[call_type]['color'] = 'rgba(0, 0, 255, 0.7)'
                edge_traces_by_type[call_type]['style'] = 'dash'
                edge_traces_by_type[call_type]['width'] = 2
            elif call_type == 'STATICCALL':
                edge_traces_by_type[call_type]['color'] = 'rgba(0, 128, 0, 0.7)'
                edge_traces_by_type[call_type]['style'] = 'dot'
                edge_traces_by_type[call_type]['width'] = 1.5
            else:
                edge_traces_by_type[call_type]['color'] = 'rgba(128, 128, 128, 0.7)'
                edge_traces_by_type[call_type]['style'] = 'solid'
                edge_traces_by_type[call_type]['width'] = 1

        x0, y0 = pos[source]
        x1, y1 = pos[target]

        edge_traces_by_type[call_type]['x'].extend([x0, x1, None])
        edge_traces_by_type[call_type]['y'].extend([y0, y1, None])

        # Create full hover info
        hover_text = (
            f"<b>From:</b> {source_data['from']}<br>"
            f"<b>To:</b> {target_data['to']}<br>"
            f"<b>Type:</b> {target_data['type']}<br>"
            f"<b>Depth:</b> {target_data['depth']}<br>"
        )

        if target_data['value_eth'] > 0:
            hover_text += f"<b>Value:</b> {target_data['value_eth']} ETH<br>"

        hover_text += f"<b>Gas:</b> {target_data['gasUsed']:,}<br>"

        if target_data['is_pyusd']:
            hover_text += f"<b>Contract:</b> {target_data['contract']}<br>"

        if target_data.get('error'):
            hover_text += f"<b style='color:red'>Error:</b> {target_data['error']}<br>"

        edge_traces_by_type[call_type]['text'].append(hover_text)

    # Create scatter traces for each call type
    edge_traces = []
    for call_type, trace_data in edge_traces_by_type.items():
        edge_traces.append(
            go.Scatter(
                x=trace_data['x'],
                y=trace_data['y'],
                line=dict(
                    width=trace_data['width'],
                    color=trace_data['color'],
                    dash=trace_data['style']
                ),
                hoverinfo='text',
                text=trace_data['text'],
                mode='lines',
                name=call_type
            )
        )

    # Create node trace with improved hover information
    node_x = []
    node_y = []
    node_colors = []
    node_sizes = []
    node_text = []  # Function name displayed on node
    hover_texts = []

    for node in G.nodes():
        node_data = G.nodes[node]
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Create more detailed text for nodes
        function_name = get_function_description(
            node_data.get('input_preview', '0x'),
            node_data['is_pyusd'],
            node_data.get('contract', 'Other')
        )

        # Short text for display (truncated for space)
        short_name = function_name.split('(')[0] if '(' in function_name else function_name
        if len(short_name) > 12:
            short_name = short_name[:10] + '...'
        node_text.append(short_name)

        # Create detailed hover text
        hover_text = f"<b style='font-size:12px'>{node_data['type']} Call</b><br>"
        hover_text += f"<b>From:</b> {node_data['from']}<br>"
        hover_text += f"<b>To:</b> {node_data['to']}<br>"
        hover_text += f"<b>Depth:</b> {node_data['depth']}<br>"

        if node_data['value_eth'] > 0:
            hover_text += f"<b>Value:</b> {node_data['value_eth']} ETH<br>"

        hover_text += f"<b>Gas Used:</b> {node_data['gasUsed']:,}<br>"

        if node_data['is_pyusd']:
            hover_text += f"<b>PYUSD Contract:</b> {node_data['contract']}<br>"
            hover_text += f"<b>Function Category:</b> {node_data['function_category']}<br>"

        hover_text += f"<b>Function:</b> {function_name}<br>"

        if node_data.get('error'):
            hover_text += f"<b style='color:red'>Error:</b> {node_data['error']}<br>"

        hover_texts.append(hover_text)

        # Node color based on contract type and error status
        if node_data.get('error'):
            node_colors.append('rgba(255, 99, 71, 0.9)')  # tomato for errors
        elif node_data['is_pyusd']:
            if "PYUSD Token" in node_data.get('contract', ''):
                node_colors.append('rgba(144, 238, 144, 0.9)')  # palegreen
            elif "Supply Control" in node_data.get('contract', ''):
                node_colors.append('rgba(135, 206, 250, 0.9)')  # lightskyblue
            else:
                node_colors.append('rgba(224, 255, 255, 0.9)')  # lightcyan
        else:
            # Gradient based on depth for non-PYUSD calls
            intensity = min(95, max(70, 95 - node_data['depth'] * 5))
            rgb_val = intensity / 100.0
            node_colors.append(f'rgba({int(rgb_val*255)}, {int(rgb_val*255)}, {int(rgb_val*255)}, 0.9)')

        # Size based on gas used
        gas_used = node_data['gasUsed']
        # Scale node size based on gas used (within reasonable bounds)
        size = max(15, min(40, 15 + (gas_used / 50000)))
        node_sizes.append(size)

    # Create node trace
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,
        textposition="top center",
        hovertext=hover_texts,
        marker=dict(
            showscale=False,
            color=node_colors,
            size=node_sizes,
            line=dict(width=1, color='#000')
        ),
        textfont=dict(
            family="Arial",
            size=9,
            color="#333"
        )
    )

    # Create legend traces for node colors
    node_color_legend = [
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(144, 238, 144, 0.9)'),
            name='PYUSD Token',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(135, 206, 250, 0.9)'),
            name='Supply Control',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(211, 211, 211, 0.9)'),
            name='Other Contract',
            showlegend=True
        ),
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color='rgba(255, 99, 71, 0.9)'),
            name='Error',
            showlegend=True
        )
    ]

    # Create figure with all traces
    fig = go.Figure()

    # Add traces
    for edge_trace in edge_traces:
        fig.add_trace(edge_trace)

    fig.add_trace(node_trace)

    # Add legend traces
    for legend_trace in node_color_legend:
        fig.add_trace(legend_trace)

    # Style the figure
    fig.update_layout(
        title='<b>Detailed Call Graph Visualization</b>',
        titlefont=dict(size=16),
        showlegend=True,
        legend=dict(
            title="Call Types",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(255, 255, 255, 0.8)"
        ),
        hovermode='closest',
        margin=dict(b=40, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white",
        height=700,
        paper_bgcolor='rgba(255,255,255,0.8)',
        plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig

# New function for creating interactive Plotly PYUSD Flow Graph
def create_plotly_flow_graph(transfers):
    """Creates an interactive Plotly Network graph for PYUSD token flows"""
    if not transfers:
        return None

    # Create a directed graph for token flows
    G = nx.DiGraph()

    # Track total transfer amount per edge for aggregation
    transfer_totals = {}

    # Calculate aggregate transfers
    for transfer in transfers:
        from_addr = transfer['from']
        to_addr = transfer['to']
        amount = transfer['amount']

        edge_key = (from_addr, to_addr)
        if edge_key in transfer_totals:
            transfer_totals[edge_key] += amount
        else:
            transfer_totals[edge_key] = amount

    # Add nodes and edges to networkx graph
    for (from_addr, to_addr), total_amount in transfer_totals.items():
        # Add nodes if they don't exist
        if from_addr not in G:
            G.add_node(from_addr, address=from_addr, label=shorten_address(from_addr))

        if to_addr not in G:
            G.add_node(to_addr, address=to_addr, label=shorten_address(to_addr))

        # Add edge with amount
        G.add_edge(from_addr, to_addr, amount=total_amount, label=format_value_pyusd(total_amount))

    # Calculate layout
    pos = nx.spring_layout(G, k=1.0, seed=42)

    # Create edge trace
    edge_x = []
    edge_y = []
    edge_text = []
    edge_amount_texts = []

    for edge in G.edges(data=True):
        source, target, data = edge

        x0, y0 = pos[source]
        x1, y1 = pos[target]

        # Calculate midpoint for displaying amount
        mid_x = (x0 + x1) / 2
        mid_y = (y0 + y1) / 2

        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

        amount_str = format_value_pyusd(data['amount'])
        edge_text.append(f"Transfer: {amount_str}<br>From: {shorten_address(source)}<br>To: {shorten_address(target)}")
        edge_amount_texts.append(amount_str)

    # Create edge trace with arrows
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='rgba(50, 150, 50, 0.8)'),
        hoverinfo='text',
        text=edge_text,
        mode='lines',
        name='Transfer'
    )

    # Create a separate trace for each edge label (amount)
    edge_label_traces = []
    edge_idx = 0
    for edge in G.edges(data=True):
        source, target, data = edge
        x0, y0 = pos[source]
        x1, y1 = pos[target]

        # Calculate midpoint for the label
        mid_x = (x0 + x1) / 2
        mid_y = (y0 + y1) / 2

        # Add label trace
        edge_label_traces.append(
            go.Scatter(
                x=[mid_x],
                y=[mid_y],
                text=[format_value_pyusd(data['amount'])],
                mode='text',
                hoverinfo='none',
                showlegend=False,
                textfont=dict(
                    size=10,
                    color='darkgreen'
                )
            )
        )

        # Add arrow trace (small marker at the target end)
        # Calculate the position for the arrow (slightly before the target)
        arrow_ratio = 0.8  # How far along the edge to place the arrow (0.8 = 80% of the way to target)
        arrow_x = x0 + (x1 - x0) * arrow_ratio
        arrow_y = y0 + (y1 - y0) * arrow_ratio

        edge_label_traces.append(
            go.Scatter(
                x=[arrow_x],
                y=[arrow_y],
                mode='markers',
                marker=dict(
                    symbol='triangle-right',
                    size=12,
                    color='rgba(50, 150, 50, 0.8)',
                    angle=45
                ),
                hoverinfo='none',
                showlegend=False
            )
        )

        edge_idx += 1

    # Create node trace with full addresses
    node_x = []
    node_y = []
    node_text = []
    node_hover = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Use full address for display
        node_text.append(node)

        # Create hover text
        node_hover.append(f"<b>Address:</b> {node}")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,
        textposition="bottom center",
        hovertext=node_hover,
        marker=dict(
            color='rgba(144, 238, 144, 0.8)',  # palegreen
            size=25,
            line=dict(width=1, color='darkgreen'),
            symbol='circle'
        ),
        textfont=dict(
            family="monospace",
            size=9,
            color="black"
        ),
        name='Address'
    )

    # Create figure with styled layout
    fig = go.Figure()

    # Add all traces
    fig.add_trace(edge_trace)
    for label_trace in edge_label_traces:
        fig.add_trace(label_trace)
    fig.add_trace(node_trace)

    # Style the figure
    fig.update_layout(
        title='<b>PYUSD Token Flow Analysis</b>',
        titlefont=dict(size=16),
        showlegend=True,
        legend=dict(
            title="Elements",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor="rgba(255, 255, 255, 0.8)"
        ),
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=60),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        template="plotly_white",
        height=600,
        paper_bgcolor='rgba(255,255,255,0.8)',
        plot_bgcolor='rgba(255,255,255,0.8)'
    )

    return fig

def parse_call_trace(trace_result, tx_hash):
    """Parses the output of callTracer and generates insights & visualizations."""
    if not trace_result:
        console.print("[warning]No trace result provided for parsing.", style="warning")
        return None, None, None, None, None, None, None, None

    # --- Basic Info Panel ---
    to_address = trace_result.get('to', 'N/A').lower()
    is_pyusd_tx = to_address in PYUSD_CONTRACTS
    pyusd_label = f"[bold green]({PYUSD_CONTRACTS[to_address]})[/bold green]" if is_pyusd_tx else ""

    # Extract gas metrics for analysis
    gas_used = int(trace_result.get('gasUsed', '0x0'), 16) if isinstance(trace_result.get('gasUsed', '0x0'), str) and trace_result.get('gasUsed', '0x0').startswith('0x') else int(trace_result.get('gasUsed', 0))

    overview_text = f"""
      [bold]Trace Summary for {shorten_address(tx_hash)}[/bold] {pyusd_label}
      Type: {trace_result.get('type', 'N/A')}
      From: {shorten_address(trace_result.get('from', 'N/A'))}
      To: {shorten_address(trace_result.get('to', 'N/A'))}
      Value: {format_value_eth(trace_result.get('value', '0x0'))}
      Gas Used: {format_gas(trace_result.get('gasUsed', '0x0'))} ({gas_used:,} units)
      Status: [bold red]Error: {trace_result['error']}[/bold red]""" if 'error' in trace_result else "[bold green]Success[/bold green]"
    console.print(Panel(overview_text, title="Trace Overview", border_style="cyan3", expand=False))

    node_counter = 0
    call_data_list = []
    state_changes = []

    # Track PYUSD transfer data for visualization
    pyusd_transfers = []

    # Track contract interactions for relationship mapping
    contract_interactions = set()

    # Track gas usage by category
    gas_by_category = {category: 0 for category in GAS_CATEGORIES.keys()}

    def add_nodes_edges(call, parent_node_id=None, depth=0, parent_addr=None):
        nonlocal node_counter
        current_node_id = f"node_{node_counter}"
        node_counter += 1

        call_type = call.get('type', 'N/A').upper()

        # Extract call data for label
        from_addr = call.get('from', 'N/A')
        from_addr_short = shorten_address(from_addr)
        to_addr = call.get('to', 'N/A')
        to_addr_lower = to_addr.lower() if to_addr else ''
        to_addr_short = shorten_address(to_addr)

        # Track contract interaction
        if parent_addr and to_addr_lower:
            contract_interactions.add((parent_addr.lower(), to_addr_lower))

        # Check for PYUSD contracts
        is_pyusd_call = to_addr_lower in PYUSD_CONTRACTS
        contract_name = PYUSD_CONTRACTS.get(to_addr_lower, None)

        # function signature detection for PYUSD calls
        input_data = call.get('input', '0x')
        function_category = "other"
        if input_data != '0x':
            method_sig = input_data[:10]

            if method_sig in PYUSD_SIGNATURES:
                function_info = PYUSD_SIGNATURES[method_sig]
                function_name = function_info["name"]
                function_category = function_info["category"]

                # Process specific functions for deeper analysis
                if is_pyusd_call and "PYUSD Token" in contract_name:
                    if method_sig == '0xa9059cbb':  # transfer
                        try:
                            # Extract params
                            to_offset = 10
                            to_param = "0x" + input_data[to_offset+24:to_offset+64]
                            amount_offset = 74
                            amount = int(input_data[amount_offset:amount_offset+64], 16)
                            pyusd_transfers.append({
                                'from': from_addr,
                                'to': to_param,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                            # Track state change
                            state_changes.append({
                                'contract': contract_name,
                                'function': function_name,
                                'type': 'transfer',
                                'from': from_addr,
                                'to': to_param,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                        except Exception:
                            pass
                    elif method_sig == '0x40c10f19':  # mint
                        try:
                            to_offset = 10
                            to_param = "0x" + input_data[to_offset+24:to_offset+64]
                            amount_offset = 74
                            amount = int(input_data[amount_offset:amount_offset+64], 16)
                            # Track state change
                            state_changes.append({
                                'contract': contract_name,
                                'function': function_name,
                                'type': 'mint',
                                'to': to_param,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                        except Exception:
                            pass
                    elif method_sig == '0x9dc29fac':  # burn
                        try:
                            from_offset = 10
                            from_param = "0x" + input_data[from_offset+24:from_offset+64]
                            amount_offset = 74
                            amount = int(input_data[amount_offset:amount_offset+64], 16)
                            # Track state change
                            state_changes.append({
                                'contract': contract_name,
                                'function': function_name,
                                'type': 'burn',
                                'from': from_addr,
                                'amount': amount,
                                'gas_used': int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
                            })
                        except Exception:
                            pass

        # Update gas usage by category - AFTER function_category is defined
        call_gas = int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
        if is_pyusd_call:
            gas_by_category[function_category] += call_gas
        else:
            gas_by_category["other"] += call_gas

        # Extract output and error data
        output_data = call.get('output', '0x')
        error_msg = call.get('error')

        # Store call data for dataframe
        try:
            gas_used_val = int(call.get('gasUsed', '0x0'), 16) if call.get('gasUsed', '0x0').startswith('0x') else int(call.get('gasUsed', 0))
            value_raw_wei = int(call.get('value', '0x0'), 16) if call.get('value', '0x0').startswith('0x') else int(call.get('value', 0))
            value_eth_float = float(w3_mainnet.from_wei(value_raw_wei, 'ether')) if w3_mainnet else (value_raw_wei / 1e18)
        except (ValueError, TypeError, AttributeError):
            gas_used_val = 0
            value_eth_float = 0.0

        # Build call info for dataframe with data
        call_info = {
            'id': current_node_id,
            'parent_id': parent_node_id,  # Track parent-child relationship
            'type': call_type,
            'depth': depth,
            'from': from_addr,
            'to': to_addr,
            'value_eth': value_eth_float,
            'gasUsed': gas_used_val,
            'is_pyusd': is_pyusd_call,
            'contract': contract_name if is_pyusd_call else "Other",
            'function_category': function_category,
            'error': error_msg,
            'input_preview': input_data[:10] + "..." if len(input_data) > 10 else input_data,
            'output_preview': output_data[:10] + "..." if len(output_data) > 10 else output_data,
        }
        call_data_list.append(call_info)

        # Process sub-calls recursively
        if 'calls' in call and isinstance(call['calls'], list):
            for sub_call in call['calls']:
                add_nodes_edges(sub_call, current_node_id, depth + 1, to_addr)

    # --- Start processing the trace from the top-level call ---
    add_nodes_edges(trace_result, depth=0)

    # Create a dataframe from collected call data
    call_df = pd.DataFrame(call_data_list)

    # Create state changes dataframe
    state_changes_df = pd.DataFrame(state_changes) if state_changes else None

    # --- Extract Logs from trace with decoding ---
    logs_data = []
    log_counter_trace = 0

    def extract_logs_recursive(call):
        nonlocal log_counter_trace
        if 'logs' in call and isinstance(call['logs'], list):
            for log in call['logs']:
                # Ensure log is a dictionary
                if not isinstance(log, dict): continue

                log_details = {
                    "address": log.get("address", "N/A"),
                    "topics": log.get("topics", []),
                    "data": log.get("data", "0x"),
                    "log_index_trace": log_counter_trace
                }
                log_counter_trace += 1

                # Check if log is from PYUSD contract
                address_lower = log_details["address"].lower()
                is_pyusd_contract = address_lower in PYUSD_CONTRACTS

                # Basic data for all logs
                log_entry = {
                    "log_idx_trace": log_details["log_index_trace"],
                    "address": log_details["address"],
                    "contract": PYUSD_CONTRACTS.get(address_lower, "Other"),
                    "is_pyusd": is_pyusd_contract,
                    "topic0": log_details["topics"][0] if log_details["topics"] else "N/A",
                    "topic0_short": log_details["topics"][0][:10]+"..." if log_details["topics"] else "N/A",
                    "details": "Not Decoded",
                    "event_name": "Unknown"
                }

                # event decoding with the registry
                if is_pyusd_contract and isinstance(log_details["topics"], list) and log_details["topics"]:
                    event_topic = log_details["topics"][0]
                    if event_topic in PYUSD_EVENTS:
                        event_info = PYUSD_EVENTS[event_topic]
                        log_entry["event_name"] = event_info["name"]

                        try:
                            # Decode event data using registered decoder
                            decoded_data = event_info["decoder"](log_details["topics"], log_details["data"])

                            # Format details based on event type
                            if "Transfer" in event_info["name"]:
                                value_pyusd_str = format_value_pyusd(decoded_data["value"])
                                log_entry["details"] = f"PYUSD Transfer: {value_pyusd_str} from {shorten_address(decoded_data['from'])} to {shorten_address(decoded_data['to'])}"
                                log_entry["is_transfer"] = True
                                log_entry["amount"] = decoded_data["value"]
                                log_entry["from_addr"] = decoded_data["from"]
                                log_entry["to_addr"] = decoded_data["to"]
                            elif "Approval" in event_info["name"]:
                                value_pyusd_str = format_value_pyusd(decoded_data["value"])
                                log_entry["details"] = f"PYUSD Approval: {shorten_address(decoded_data['owner'])} approved {value_pyusd_str} for {shorten_address(decoded_data['spender'])}"
                                log_entry["is_approval"] = True
                                log_entry["amount"] = decoded_data["value"]
                                log_entry["owner"] = decoded_data["owner"]
                                log_entry["spender"] = decoded_data["spender"]
                            elif "Paused" in event_info["name"]:
                                account = decoded_data.get("account", "N/A")
                                log_entry["details"] = f"PYUSD Paused by {shorten_address(account) if account else 'N/A'}"
                                log_entry["is_pause"] = True
                            elif "Unpaused" in event_info["name"]:
                                account = decoded_data.get("account", "N/A")
                                log_entry["details"] = f"PYUSD Unpaused by {shorten_address(account) if account else 'N/A'}"
                                log_entry["is_unpause"] = True

                        except Exception as decode_err:
                            log_entry["details"] = f"PYUSD Event (Decode Error: {decode_err})"

                logs_data.append(log_entry)

        # Check sub-calls recursively
        if 'calls' in call and isinstance(call['calls'], list):
            for sub_call in call['calls']:
                extract_logs_recursive(sub_call)

    # Extract logs if present in trace config
    extract_logs_recursive(trace_result)
    logs_df = pd.DataFrame(logs_data) if logs_data else pd.DataFrame()

    # --- Create Contract Interaction Graph with Plotly ---
    contract_graph = None
    if contract_interactions:
        try:
            contract_graph = create_plotly_contract_interaction_graph(contract_interactions)
        except Exception as viz_err:
            console.print(f"[warning]Could not create contract interaction graph: {viz_err}", style="warning")

    # --- Create Detailed Call Graph with Plotly ---
    call_graph = None
    if call_data_list:
        try:
            call_graph = create_plotly_call_graph(call_data_list)
        except Exception as viz_err:
            console.print(f"[warning]Could not create detailed call graph: {viz_err}", style="warning")

    # Create PYUSD flow graph if transfers exist
    flow_graph = None
    if pyusd_transfers:
        try:
            flow_graph = create_plotly_flow_graph(pyusd_transfers)
        except Exception as flow_err:
            console.print(f"[warning]Could not create PYUSD flow graph: {flow_err}", style="warning")

    # Create Gas Usage by Category
    gas_category_df = pd.DataFrame(
        [{"category": k, "gas_used": v} for k, v in gas_by_category.items() if v > 0]
    )

    return call_graph, call_df, logs_df, flow_graph, contract_graph, gas_category_df, state_changes_df, pyusd_transfers

# =============================================================================================
# --- Execute callTracer Analysis ---
# =============================================================================================
if 'TARGET_TX_HASH' in locals() and validate_tx_hash: # Use the validation flag from setup cell
    console.print("\n\n[bold]🎯 Using callTracer on Mainnet[/bold]", style="cyan3")
    console.print("───────────────────────────────", style="cyan3")
    console.print(f"Target Transaction : {TARGET_TX_HASH}")

    # Use the trace config from the configuration
    trace_result_call = make_rpc_request("debug_traceTransaction",
                                         [TARGET_TX_HASH, {"tracer": "callTracer", "tracerConfig": TRACE_CONFIGS["callTracer"]}],
                                         network='mainnet')

    if trace_result_call:
        console.print("[success]Successfully received trace data.", style="success")

        # --- Parse and Visualize ---
        try:
            call_graph, call_df, logs_df, pyusd_flow, contract_graph, gas_category_df, state_changes_df, pyusd_transfers = parse_call_trace(trace_result_call, TARGET_TX_HASH)

            # Create output widgets for each visualization (add this right after parsing the trace)
            contract_graph_output = widgets.Output()
            call_graph_output = widgets.Output()
            flow_graph_output = widgets.Output()

            # 1. TRANSACTION OVERVIEW DASHBOARD
            console.print("\n\n[bold cyan3]🔍 PYUSD TRANSACTION ANALYSIS DASHBOARD[/bold cyan3]", justify="left")
            console.print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━", style="cyan3", justify="left")

            if call_df is not None and not call_df.empty:
                # Basic transaction metrics
                tx_stats = {
                    "total_calls": len(call_df),
                    "pyusd_calls": len(call_df[call_df['is_pyusd']]),
                    "max_call_depth": call_df['depth'].max(),
                    "total_gas": call_df['gasUsed'].sum(),
                    "errors": len(call_df[call_df['error'].notnull()])
                }

                stats_table = Table(title="Transaction Metrics", show_header=True, header_style="bold cyan")
                stats_table.add_column("Metric", style="dim")
                stats_table.add_column("Value")

                for k, v in tx_stats.items():
                    if k == 'total_gas':
                        stats_table.add_row(k.replace('_', ' ').title(), f"{v:,} gas units")
                    else:
                        stats_table.add_row(k.replace('_', ' ').title(), str(v))

                console.print(stats_table)

           # 2. Display Contract Interaction Graph
            if contract_graph:
                console.print("\n\n[bold cyan3]📊 Contract Interaction Overview[/bold cyan3]")
                console.print("─────────────────────────────────", style="cyan3")
                try:
                    # Simply display the visualization
                    display(contract_graph)
                    console.print("\n\n[info]This graph shows the high-level interactions between contracts in this transaction.", style="info")
                except Exception as viz_err:
                    console.print(f"[warning]Could not render contract interaction graph: {viz_err}", style="warning")

            # 3. Display Call Graph Visualization
            console.print("\n\n[bold cyan3]📊 Detailed Call Graph Visualization[/bold cyan3]")
            console.print("─────────────────────────────────────", style="cyan3")
            if call_graph:
                try:
                    # Simply display the visualization
                    display(call_graph)
                    console.print("\n\n[info]This visualization shows the detailed call hierarchy in this transaction.", style="info")
                except Exception as viz_err:
                    console.print(f"[warning]Could not render visualization: {viz_err}", style="warning")
            else:
                console.print("[warning]Call graph generation failed.", style="warning")

            # 4. Display PYUSD Flow Graph if available
            if pyusd_flow:
                console.print("\n\n[bold cyan3]🔄 PYUSD Token Flow Analysis[/bold cyan3]")
                console.print("─────────────────────────────", style="cyan3")
                try:
                    # Simply display the visualization
                    display(pyusd_flow)
                    console.print("\n\n[info]This graph shows the movement of PYUSD tokens in this transaction.", style="info")
                except Exception as flow_err:
                    console.print(f"[warning]Could not render PYUSD flow: {flow_err}", style="warning")

            # 5. State Changes Analysis
            if state_changes_df is not None and not state_changes_df.empty:
                console.print("\n\n[bold cyan3]🔄 PYUSD State Changes[/bold cyan3]")
                console.print("───────────────────────", style="cyan3")
                console.print("[info]The following state changes occurred in PYUSD contracts:", style="info")

                # Format amounts in dataframe
                state_changes_df['formatted_amount'] = state_changes_df['amount'].apply(format_value_pyusd)

                # Display state changes with appropriate columns
                display(state_changes_df[['contract', 'function', 'type', 'formatted_amount', 'gas_used']])

                # Summary of state impact
                if 'type' in state_changes_df.columns:
                    changes_by_type = state_changes_df['type'].value_counts()
                    console.print("\n[bold]State Change Summary:[/bold]")
                    for change_type, count in changes_by_type.items():
                        console.print(f"- {change_type.title()}: {count} operations")
            else:
                console.print("\n\n[info]No direct PYUSD state changes detected in this transaction.", style="info")

            # 6. Gas Usage Analysis
            if gas_category_df is not None and not gas_category_df.empty:
                console.print("\n\n[bold yellow3]⛽ Gas Usage Analysis[/bold yellow3]")
                console.print("──────────────────────", style="yellow3")

                # Create simple gas usage table
                gas_table = Table(title="Gas Usage by Operation Category", show_header=True, header_style="bold yellow3")
                gas_table.add_column("Category", style="dim")
                gas_table.add_column("Gas Used", justify="right")
                gas_table.add_column("Percentage", justify="right")

                total_gas = gas_category_df['gas_used'].sum()

                for _, row in gas_category_df.iterrows():
                    category = row['category'].replace('_', ' ').title()
                    gas_used = row['gas_used']
                    percentage = (gas_used / total_gas * 100) if total_gas > 0 else 0
                    gas_table.add_row(
                        category,
                        f"{gas_used:,}",
                        f"{percentage:.1f}%"
                    )

                console.print(gas_table)

                # Gas usage visualization
                try:
                    # Color by operation category for better visibility
                    fig_gas = px.pie(gas_category_df, values='gas_used', names='category',
                                     title=f'<b>Gas Usage Distribution ({shorten_address(TARGET_TX_HASH)})</b>')
                    fig_gas.update_layout(
                        template="plotly_white",
                        title={
                            'y': 0.95,
                            'x': 0.5,
                            'xanchor': 'center',
                            'yanchor': 'top',
                            'font': {'size': 16}
                        },
                        margin=dict(t=100, b=50, l=50, r=50)  # Increased top margin
                    )
                    fig_gas.show()
                except Exception as plot_err:
                    console.print(f"[warning]Could not generate gas usage plot: {plot_err}", style="warning")

            # 7. Filtered Call Data Table
            if call_df is not None and not call_df.empty:
                console.print("\n\n[bold cyan3]📋 Key Contract Calls[/bold cyan3]")
                console.print("─────────────────────", style="cyan3")

                # Focus on PYUSD calls for a cleaner view
                pyusd_calls = call_df[call_df['is_pyusd']]

                if not pyusd_calls.empty:
                    console.print(f"[success]Found {len(pyusd_calls)} PYUSD-related calls in this transaction.", style="success")

                    # Style the DataFrame for better visibility
                    def highlight_pyusd(val):
                        return 'background-color: palegreen' if val else ''

                    # Display with conditional formatting - only important columns
                    display_cols = ['id', 'type', 'depth', 'contract', 'function_category', 'gasUsed', 'error']
                    display(pyusd_calls[display_cols])
                else:
                    console.print("[info]No PYUSD-specific calls found in this transaction.", style="info")

                # Show high gas usage calls with function names
                console.print("\n[bold yellow3]Highest Gas Usage Calls:[/bold yellow3]")
                high_gas_calls = call_df.nlargest(5, 'gasUsed')

                # Add function description to high gas calls
                high_gas_calls['function_name'] = high_gas_calls.apply(
                    lambda row: get_function_description(
                        row['input_preview'],
                        row['is_pyusd'] if 'is_pyusd' in row else False,
                        row['contract'] if 'contract' in row else "Non-PYUSD Contract"
                    ),
                    axis=1
                )

                # Display with function name instead of category if available
                display(high_gas_calls[['id', 'type', 'contract', 'function_name', 'gasUsed']])

            # 8. PYUSD Event Analysis
            if logs_df is not None and not logs_df.empty:
                console.print("\n\n[bold green3]📝 PYUSD Events Analysis[/bold green3]")
                console.print("─────────────────────────", style="green3")

                # Highlight PYUSD logs
                pyusd_logs = logs_df[logs_df['is_pyusd']]
                if not pyusd_logs.empty:
                    console.print(f"[success]Found {len(pyusd_logs)} PYUSD events in this transaction.", style="success")

                    # Group by event type
                    if 'event_name' in pyusd_logs.columns:
                        event_counts = pyusd_logs['event_name'].value_counts()

                        event_table = Table(title="PYUSD Events", show_header=True, header_style="bold green3")
                        event_table.add_column("Event Type", style="dim")
                        event_table.add_column("Count", justify="right")

                        for event, count in event_counts.items():
                            event_table.add_row(event, str(count))

                        console.print(event_table)

                    # Display detailed event data
                    console.print("\n\n[bold green3]PYUSD Event Details:[/bold green3]")
                    display(pyusd_logs[['contract', 'event_name', 'details']])

                    # Transfer value analysis
                    if 'amount' in pyusd_logs.columns and 'is_transfer' in pyusd_logs.columns:
                        transfer_logs = pyusd_logs[pyusd_logs['is_transfer']]
                        if not transfer_logs.empty:
                            total_transferred = transfer_logs['amount'].sum()
                            console.print(f"\n\n[info][bold cyan3]Total PYUSD transferred:[bold cyan3] {format_value_pyusd(total_transferred)}", style="info")
                else:
                    console.print("[info]No PYUSD events found in this transaction.", style="info")
            else:
                console.print("[info]No logs extracted from trace.", style="info")

            # 9. Add Recommendations Section
            console.print("\n\n[bold cyan3]💡 Analysis Observations & Recommendations[/bold cyan3]")
            console.print("──────────────────────────────────────────", style="cyan3")

            recommendations = []

            # Check for high gas usage patterns
            if call_df is not None and not call_df.empty:
                high_gas_threshold = call_df['gasUsed'].sum() * 0.25  # 25% of total gas
                high_gas_ops = call_df[call_df['gasUsed'] > high_gas_threshold]
                if not high_gas_ops.empty:
                    recommendations.append(f"- High gas operations detected: {len(high_gas_ops)} calls used >25% of transaction gas")

            # Check for deep call stack
            if call_df is not None and 'depth' in call_df.columns:
                max_depth = call_df['depth'].max()
                if max_depth > 5:
                    recommendations.append(f"- Deep call stack detected (max depth: {max_depth})")

            # Check token flow patterns
            if logs_df is not None and 'is_transfer' in logs_df.columns:
                transfer_count = logs_df['is_transfer'].sum()
                if transfer_count > 3:
                    recommendations.append(f"- Complex token movement detected ({transfer_count} transfers)")

            # Add general PYUSD observations
            if call_df is not None and 'is_pyusd' in call_df.columns:
                pyusd_calls = call_df[call_df['is_pyusd']]
                if not pyusd_calls.empty:
                    recommendations.append(f"- Transaction involves {len(pyusd_calls)} PYUSD contract interactions")

            # Display recommendations
            if recommendations:
                for rec in recommendations:
                    console.print(rec)
            else:
                console.print("[info]No specific observations to highlight for this transaction.", style="info")

            # 10. Export Options
            console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
            console.print("──────────────────", style="cyan3")

            # Create export output area
            export_output = widgets.Output()

            # Create export buttons with proper styling
            export_buttons = widgets.HBox([
                widgets.Button(
                    description='Export to CSV',
                    button_style='primary',  # Green
                    layout=widgets.Layout(width='150px')
                ),
                widgets.Button(
                    description='Export as JSON',
                    button_style='warning',  # Orange
                    layout=widgets.Layout(width='150px')
                ),
                widgets.Button(
                    description='Export to Google Sheets',
                    button_style='info',     # Blue
                    layout=widgets.Layout(width='200px')
                )
            ])

            # Define export handlers with simplified loading indicators
            def export_csv(b):
                with export_output:
                    clear_output()
                    console.print("[cyan3]Exporting to CSV...", style="info")

                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"calltrace_{TARGET_TX_HASH[:10]}_{timestamp}.csv"

                    try:
                        # Export to CSV
                        result = download_csv_direct(call_df, filename)

                        # Clear loading message and show success
                        clear_output()
                        console.print("✓ Successfully exported to CSV", style="spring_green3")
                        display(result)
                    except Exception as e:
                        clear_output()
                        console.print(f"❌ Error exporting to CSV: {str(e)}", style="error")

            def export_json(b):
                with export_output:
                    clear_output()
                    console.print("[cyan3]Exporting to JSON...", style="info")

                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"calltrace_{TARGET_TX_HASH[:10]}_{timestamp}.json"

                    try:
                        # Prepare export data
                        export_data = {
                            "transaction_hash": TARGET_TX_HASH,
                            "analysis_type": "callTracer",
                            "transaction_stats": tx_stats if 'tx_stats' in locals() else {},
                            "state_changes": state_changes_df.to_dict('records') if state_changes_df is not None and not state_changes_df.empty else [],
                            "gas_by_category": gas_category_df.to_dict('records') if gas_category_df is not None and not gas_category_df.empty else [],
                            "pyusd_transfers": pyusd_transfers if pyusd_transfers is not None else []
                        }

                        # Export to JSON
                        result = download_json_direct(export_data, filename)

                        # Clear loading message and show success
                        clear_output()
                        console.print("✓ Successfully exported to JSON", style="spring_green3")
                        display(result)
                    except Exception as e:
                        clear_output()
                        console.print(f"❌ Error exporting to JSON: {str(e)}", style="error")

            def export_to_sheets(b):
                with export_output:
                    clear_output()
                    try:
                        # Collect all data for the sheet
                        export_data = {
                            "transaction_hash": TARGET_TX_HASH,
                            "transaction_stats": tx_stats if 'tx_stats' in locals() else {},
                            "gas_distribution": gas_category_df.to_dict('records') if gas_category_df is not None else [],
                            "pyusd_transfers": pyusd_transfers if pyusd_transfers is not None else [],
                            "state_changes": state_changes_df.to_dict('records') if state_changes_df is not None and not state_changes_df.empty else [],
                            "logs_df": logs_df if logs_df is not None and not logs_df.empty else None
                        }

                        # Call the export function
                        spreadsheet_url, sheet_title = export_to_google_sheets(call_df, export_data, TARGET_TX_HASH)

                        # Open the spreadsheet and display link
                        display(Javascript(f'window.open("{spreadsheet_url}", "_blank");'))

                        # 2. Display *only* the link and success message as static HTML output
                        clear_output(wait=True)
                        console.print("✓ Successfully exported to Google Sheets", style="spring_green3")
                        display(HTML(f'''
                        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
                        '''))

                    except Exception as e:
                        clear_output()
                        console.print(f"❌ Error exporting to Google Sheets: {str(e)}", style="error")
                        html = f"<div style='color:red'>Error exporting to Google Sheets: {str(e)}</div>"
                        display(HTML(html))

                        # Fallback to CSV
                        console.print("[cyan3]Falling back to CSV download...", style="info")
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = f"calltrace_{TARGET_TX_HASH[:10]}_{timestamp}.csv"
                        result = download_csv_direct(call_df, filename)

                        clear_output()
                        console.print("✓ CSV fallback ready", style="spring_green3")
                        display(result)
                        display(HTML("<div>Falling back to CSV download due to Google Sheets error.</div>"))

            # Connect handlers to buttons
            export_buttons.children[0].on_click(export_csv)
            export_buttons.children[1].on_click(export_json)
            export_buttons.children[2].on_click(export_to_sheets)

            # Display button container and output area
            display(export_buttons)
            display(export_output)

        except Exception as parse_err:
            console.print(f"[error]Failed during parsing or visualization: {parse_err}", style="error")
            import traceback
            console.print(traceback.format_exc())
    else:
        console.print("[error]Failed to retrieve trace data from RPC node.", style="error")
else:
    console.print("[warning]No valid transaction hash found. Please set TARGET_TX_HASH to a valid transaction hash.", style="warning")

### 1.2.2 Using `structLog` Tracer: Opcode-Level Execution Analysis (Use Cautiously)

The `structLog` tracer dives deep into the Ethereum Virtual Machine (EVM), providing a step-by-step log of each opcode executed during the transaction. For each step, it details the program counter (PC), opcode, remaining gas, gas cost, stack contents, memory state, and storage changes (if enabled).

> **⚠️ Extreme Granularity & Resource Intensity**
>
> *   **Method:** `debug_traceTransaction` with `tracer: "structLog"`
> *   **Multiplier:** `50x` (Same base multiplier as `callTracer`, but output size is *significantly* larger)
> *   **Output Size:** Can generate **very large** outputs (potentially hundreds of thousands of steps/lines, consuming significant memory/browser resources) for even moderately complex transactions.
> *   **Use Case:** Best suited for highly specific debugging tasks, deep gas optimization analysis at the opcode level, or verifying precise execution paths, rather than general transaction overview. **Use with caution.**

> **🚀 Leveraging GCP's Premium RPC Capabilities**
>
> *   **GCP Advantage:** While the base multiplier is `50x`, the sheer volume of data returned by `structLog` makes it extremely demanding on RPC node resources. GCP's infrastructure and generous quotas make it feasible to retrieve these detailed traces where other providers might time out, restrict output size, or charge heavily.
> *   **PYUSD Insight:** `structLog` can be used for:
>     *   **Fine-grained Gas Profiling:** Identifying exactly which opcodes consume the most gas during PYUSD function execution (e.g., `SSTORE` during transfers/approvals).
>     *   **Debugging Reverts:** Pinpointing the exact opcode and state (stack/memory) where a PYUSD transaction failed.
>     *   **Security Analysis:** Examining low-level execution for potential vulnerabilities or unexpected behavior within PYUSD or interacting contracts.

**Analysis Workflow:**

1.  **Conditional Execution:** The code includes a flag (`RUN_STRUCTLOG_TRACE`) which defaults to `False`. Set this to `True` only if you specifically need this level of detail.
2.  **Fetch Trace:** Calls `debug_traceTransaction` using `TARGET_TX_HASH` and the `structLog` configuration.
3.  **Parse Steps:** The `parse_struct_log` function processes the potentially massive list of execution steps.
4.  **Summarize & Visualize:**
    *   **Overview:** Displays total steps, gas cost, call depth, and highlights PYUSD execution percentage.
    *   **Gas Analysis:** Generates plots showing gas cost distribution by opcode *category* and by individual *opcode*.
    *   **Execution Timeline:** Plots gas remaining over execution steps, highlighting sections executed within PYUSD contracts.
    *   **PYUSD Focus:** Analyzes opcode frequency and gas usage specifically within PYUSD contract execution steps.
    *   **Data Sample:** Displays the first few steps of the detailed execution trace DataFrame.
    *   **Export Options:** Allows downloading the full (potentially large) execution step data.

**💡 What to Look For:**
*   **Gas Plots:** Identify which opcode categories (e.g., `STORAGE`, `MEMORY`, `CALL`) and specific opcodes (e.g., `SSTORE`, `CALL`, `KECCAK256`) dominate gas consumption. Compare this within PYUSD vs. non-PYUSD sections.
*   **Execution Timeline:** Observe gas depletion patterns. Look for sharp drops corresponding to expensive operations. Note the percentage of time spent executing PYUSD code.
*   **Data Sample:** Understand the structure of the per-opcode data (PC, gas, stack, memory).
*   **(If Debugging):** Search the full trace data (if exported) for `REVERT` opcodes or unexpected stack/memory states near the point of failure.

In [ ]:
# =============================================================================================
# 🔬 Trace Transaction using debug_traceTransaction (structLog Tracer Opcode-Level Detail)
# =============================================================================================
# This cell performs detailed opcode-level transaction analysis using the debug_traceTransaction method with the structLog tracer.
# It prepares the data and presents insights by:
# - Conditionally executing the trace based on the RUN_STRUCTLOG_TRACE flag.
# - Parsing the raw structLog output, calculating step-by-step gas costs, categorizing opcodes, and tracking contract execution context (including PYUSD).
# - Generating interactive Plotly visualizations for overall gas usage (by category, by opcode), execution timeline (highlighting PYUSD), and PYUSD-specific gas analysis.
# - Displaying summary statistics (Panel, Rich tables) and a sample of the processed trace data.
# - Providing interactive buttons for exporting the detailed trace data to CSV, JSON, or Google Sheets using helper functions.

import base64
import json
from datetime import datetime
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output

def download_csv_direct(df, filename=None):
    """Creates a direct download for CSV without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.csv"

    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:text/csv;base64,{payload}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def download_json_direct(data, filename=None):
    """Creates a direct download for JSON without intermediate display."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"pyusd_data_{timestamp}.json"

    # Convert to JSON string (handling non-serializable objects)
    json_str = json.dumps(data, default=str, indent=2)
    b64 = base64.b64encode(json_str.encode()).decode()

    # Create direct download HTML
    html = f'''
    <script>
    function download(filename, data) {{
        const a = document.createElement('a');
        a.href = data;
        a.download = filename;
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }}

    download("{filename}", "data:application/json;base64,{b64}");
    </script>
    <div>Downloading {filename}...</div>
    '''
    return HTML(html)

def export_to_google_sheets(df, data_dict, tx_hash):
    """Export StructLog analysis data to Google Sheets with rich formatting."""
    # Show loading message
    console.print("[cyan3]Exporting to Google Sheets...", style="info")

    try:
        # Create a new Google Sheet with meaningful title
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_title = f"PYUSD StructLog Analysis {tx_hash[:10]} {timestamp}"

        # Use the global gc_sheets client that's already authenticated
        spreadsheet = gc_sheets.create(sheet_title)

        # Get the default worksheet and rename it
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update_title("Execution Steps")

        # Set up a header with transaction info
        header_values = [
            ["PYUSD StructLog Analysis"],
            [f"Transaction: {tx_hash}"],
            [f"Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"],
            [""],  # Empty row for spacing
        ]
        worksheet.update("A1", header_values)

        # Format the header with bold text and colored background
        worksheet.format("A1:A1", {
            "textFormat": {"bold": True, "fontSize": 14},
            "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 1.0}
        })

        worksheet.format("A2:A3", {
            "textFormat": {"bold": True, "fontSize": 12}
        })

        current_row = 5  # Start after header

        # Add transaction stats summary
        if "summary" in data_dict:
            stats = data_dict["summary"]

            # Add section title
            worksheet.update(f"A{current_row}", [["Analysis Summary"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.9, "blue": 1.0}
            })
            current_row += 1

            # Add stats data
            stats_rows = []
            stats_rows.append(["Metric", "Value"])  # Header row
            for key, value in stats.items():
                # Format keys and values appropriately
                formatted_key = key.replace("_", " ").title()

                # Try to format numerical values with commas
                try:
                    if isinstance(value, (int, float)):
                        formatted_value = f"{value:,}"
                    else:
                        formatted_value = str(value)
                except:
                    formatted_value = str(value)

                stats_rows.append([formatted_key, formatted_value])

            # Add stats table
            stats_start_row = current_row
            worksheet.update(f"A{stats_start_row}", stats_rows)

            # Format stats table header
            worksheet.format(f"A{stats_start_row}:B{stats_start_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            current_row += len(stats_rows) + 2  # Add extra space after table

        # Add opcode distribution reference
        worksheet.update(f"A{current_row}", [["Opcode Distribution Analysis"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 0.7, "green": 0.9, "blue": 1.0}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Opcode distribution visualizations are available in the notebook"]])
        current_row += 2

        # Add gas usage reference
        worksheet.update(f"A{current_row}", [["Gas Usage Analysis"]])
        worksheet.format(f"A{current_row}:A{current_row}", {
            "textFormat": {"bold": True, "fontSize": 12},
            "backgroundColor": {"red": 1.0, "green": 0.9, "blue": 0.7}
        })
        current_row += 1

        worksheet.update(f"A{current_row}", [["📊 Gas usage visualizations are available in the notebook"]])
        current_row += 2

        # Add main DataFrame data
        if not df.empty:
            # Add a section title
            worksheet.update(f"A{current_row}", [["Execution Steps Data"]])
            worksheet.format(f"A{current_row}:A{current_row}", {
                "textFormat": {"bold": True, "fontSize": 12},
                "backgroundColor": {"red": 0.8, "green": 0.8, "blue": 1.0}
            })
            current_row += 1

            # For StructLog data, select most important columns for readability
            # if dataframe has too many columns
            if len(df.columns) > 10:
                key_cols = ["pc", "op", "gas", "gasCost", "depth", "stack", "memory", "storage"]
                display_cols = [col for col in key_cols if col in df.columns]

                # Add any custom columns that might contain analysis results
                other_important_cols = []
                for col in df.columns:
                    if col not in display_cols and any(x in col.lower() for x in ["pyusd", "token", "contract", "note", "category"]):
                        other_important_cols.append(col)

                display_cols.extend(other_important_cols)
                display_df = df[display_cols]
            else:
                display_df = df

            # Convert DataFrame to list of lists for the worksheet
            df_values = [display_df.columns.tolist()] + display_df.values.tolist()

            # Format values for better readability
            for i in range(1, len(df_values)):
                for j, col in enumerate(display_df.columns):
                    val = df_values[i][j]

                    # Format different column types appropriately
                    if pd.isnull(val):
                        df_values[i][j] = ""
                    elif col in ["gas", "gasCost"] and isinstance(val, (int, float)):
                        df_values[i][j] = f"{val:,}"
                    elif col in ["stack", "memory", "storage"] and isinstance(val, (list, dict)):
                        # Truncate long data structures to prevent huge cells
                        df_values[i][j] = str(val)[:100] + "..." if len(str(val)) > 100 else str(val)
                    else:
                        df_values[i][j] = str(val)

            worksheet.update(f"A{current_row}", df_values)

            # Format the DataFrame header
            worksheet.format(f"A{current_row}:{chr(65+len(display_df.columns)-1)}{current_row}", {
                "textFormat": {"bold": True},
                "backgroundColor": {"red": 0.95, "green": 0.95, "blue": 0.95}
            })

            # Add alternating row colors for readability
            data_rows = len(df_values)
            for i in range(2, data_rows + 1, 2):
                row_num = current_row + i - 1
                worksheet.format(f"A{row_num}:{chr(65+len(display_df.columns)-1)}{row_num}", {
                    "backgroundColor": {"red": 0.97, "green": 0.97, "blue": 1.0}
                })

        # Try to auto-resize columns for better readability
        try:
            worksheet.columns_auto_resize(0, 10)  # Attempt to resize first 10 columns
        except:
            pass  # Ignore if not supported

        # Clear loading message and show success message
        clear_output()
        console.print("✓ Successfully exported to Google Sheets", style="spring_green3")

        # Open the spreadsheet in a new tab
        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{spreadsheet.id}"
        html = f'''
        <script>
        window.open("{spreadsheet_url}", "_blank");
        </script>
        <div>Spreadsheet created and opened: <a href="{spreadsheet_url}" target="_blank">{sheet_title}</a></div>
        '''
        return HTML(html)

    except Exception as e:
        # Clear loading message and show error
        clear_output()
        console.print(f"❌ Error creating Google Sheet: {str(e)}", style="error")
        return HTML(f"<div style='color:red'>Error creating Google Sheet: {str(e)}</div>")

def parse_struct_log(struct_logs_list, tx_hash):
    """Parses structLog output for analysis and visualization."""
    if not struct_logs_list or not isinstance(struct_logs_list, list):
        console.print("[warning]No structLog data provided or invalid format.", style="warning")
        return None

    # --- Initialize tracking variables ---
    log_data = []
    total_gas_cost = 0
    last_gas = 0
    current_contracts = {}  # Map depth -> contract address
    pyusd_execution_steps = 0

    # --- Define OPCODE categories for better analysis ---
    OPCODE_CATEGORIES = {
        "arithmetic": ["ADD", "MUL", "SUB", "DIV", "SDIV", "MOD", "SMOD", "ADDMOD", "MULMOD", "EXP", "SIGNEXTEND"],
        "comparison": ["LT", "GT", "SLT", "SGT", "EQ", "ISZERO"],
        "bitwise": ["AND", "OR", "XOR", "NOT", "BYTE", "SHL", "SHR", "SAR"],
        "memory": ["MLOAD", "MSTORE", "MSTORE8", "MSIZE", "MCOPY"],
        "storage": ["SLOAD", "SSTORE"],
        "flow": ["JUMP", "JUMPI", "JUMPDEST", "PC", "STOP", "RETURN", "REVERT"],
        "stack": ["POP", "PUSH1", "PUSH2", "PUSH3", "PUSH4", "PUSH5", "PUSH6", "PUSH7", "PUSH8",
                 "PUSH9", "PUSH10", "PUSH11", "PUSH12", "PUSH13", "PUSH14", "PUSH15", "PUSH16",
                 "PUSH17", "PUSH18", "PUSH19", "PUSH20", "PUSH21", "PUSH22", "PUSH23", "PUSH24",
                 "PUSH25", "PUSH26", "PUSH27", "PUSH28", "PUSH29", "PUSH30", "PUSH31", "PUSH32",
                 "DUP1", "DUP2", "DUP3", "DUP4", "DUP5", "DUP6", "DUP7", "DUP8",
                 "DUP9", "DUP10", "DUP11", "DUP12", "DUP13", "DUP14", "DUP15", "DUP16",
                 "SWAP1", "SWAP2", "SWAP3", "SWAP4", "SWAP5", "SWAP6", "SWAP7", "SWAP8",
                 "SWAP9", "SWAP10", "SWAP11", "SWAP12", "SWAP13", "SWAP14", "SWAP15", "SWAP16"],
        "environment": ["ADDRESS", "BALANCE", "ORIGIN", "CALLER", "CALLVALUE", "CALLDATALOAD",
                        "CALLDATASIZE", "CALLDATACOPY", "CODESIZE", "CODECOPY", "GASPRICE",
                        "EXTCODESIZE", "EXTCODECOPY", "RETURNDATASIZE", "RETURNDATACOPY",
                        "EXTCODEHASH", "BLOCKHASH", "COINBASE", "TIMESTAMP", "NUMBER",
                        "DIFFICULTY", "GASLIMIT", "CHAINID", "SELFBALANCE", "BASEFEE"],
        "contract": ["CREATE", "CREATE2", "CALL", "CALLCODE", "DELEGATECALL", "STATICCALL", "SELFDESTRUCT"],
        "logging": ["LOG0", "LOG1", "LOG2", "LOG3", "LOG4"],
        "gas": ["GAS"],
        "other": []
    }

    def get_opcode_category(opcode):
        """Determine the category of an opcode based on predefined categories."""
        for category, opcodes in OPCODE_CATEGORIES.items():
            if opcode in opcodes:
                return category
        return "other"

    console.print(f"[info]Parsing {len(struct_logs_list):,} structLog steps for {shorten_address(tx_hash)}\n\n", style="info")

    # Get initial gas from the first step if available
    if struct_logs_list and isinstance(struct_logs_list[0], dict) and 'gas' in struct_logs_list[0]:
        last_gas = struct_logs_list[0].get('gas', 0)

    # --- Process each execution step in the struct logs ---
    for i, step in enumerate(struct_logs_list):
        # Ensure step is a dictionary
        if not isinstance(step, dict):
            continue

        # Calculate gas cost for this step
        current_gas = step.get('gas', last_gas)
        gas_cost = last_gas - current_gas
        total_gas_cost += gas_cost if gas_cost > 0 else 0

        # Get basic step information
        depth = step.get('depth', 0)
        op = step.get('op', 'N/A')

        # Track contract context changes on CALL instructions
        is_pyusd_related = False
        if op in ['CALL', 'STATICCALL', 'DELEGATECALL']:
            try:
                stack = step.get('stack', [])
                if len(stack) >= 2:  # Need at least 2 stack items for call address
                    # Address is the second stack item for CALL, STATICCALL
                    address_raw = stack[1]
                    if address_raw.startswith('0x'):
                        address = '0x' + address_raw[-40:]
                    else:
                        # Handle numeric representation
                        try:
                            address = '0x' + hex(int(address_raw, 16))[-40:].zfill(40)
                        except ValueError:
                            address = None

                    if address:
                        current_contracts[depth+1] = address.lower()
                        is_pyusd_related = is_pyusd_contract(address)
            except Exception:
                pass

        # Check if we're currently in a PYUSD contract
        current_contract = current_contracts.get(depth, None)
        is_in_pyusd = is_pyusd_contract(current_contract) if current_contract else False

        if is_in_pyusd:
            pyusd_execution_steps += 1

        # Categorize the opcode
        opcode_category = get_opcode_category(op)

        # Build the execution data record
        log_data.append({
            'step': i,
            'pc': step.get('pc', 0),
            'op': op,
            'opcode_category': opcode_category,
            'gas': current_gas,
            'gasCost': gas_cost if gas_cost >= 0 else 0,  # Ensure non-negative cost
            'depth': depth,
            'stack_depth': len(step.get('stack', [])),
            'mem_size_bytes': len(step.get('memory', [])) * 32,  # Memory size in bytes
            'current_contract': current_contract,
            'is_pyusd_contract': is_in_pyusd,
            'is_pyusd_related': is_pyusd_related or is_in_pyusd
        })
        last_gas = current_gas

    if not log_data:
        console.print("[warning]No valid steps found in structLog data after parsing.", style="warning")
        return None

    # --- Convert to DataFrame for analysis ---
    df = pd.DataFrame(log_data)

    # --- Display Summary ---
    pyusd_percentage = (pyusd_execution_steps / len(df) * 100) if len(df) > 0 else 0

    console.print(Panel(f"""
[bold]structLog Trace Summary for {shorten_address(tx_hash)}[/bold]
Total Steps Parsed: {len(df):,}
Total Gas Cost (calc): {total_gas_cost:,}
Max Depth: {df['depth'].max() if not df.empty else 'N/A'}
Max Stack Depth: {df['stack_depth'].max() if not df.empty else 'N/A'}
Max Memory (bytes): {df['mem_size_bytes'].max() if not df.empty else 'N/A'}
Steps in PYUSD Contracts: {pyusd_execution_steps:,} ({pyusd_percentage:.1f}% of execution)""",
        title="structLog Overview", border_style="cyan3", expand=False))

    # --- Generate Visualizations ---
    if not df.empty:
        # --- Plot Gas Cost by Opcode Category ---
        try:
            # Visualization Header
            console.print("\n\n[bold yellow3]⛽ Gas Cost by Opcode Category[/bold yellow3]")
            console.print("────────────────────────────────", style="yellow3")

            # Calculate opcode category gas costs
            category_gas = df.groupby('opcode_category')['gasCost'].sum().reset_index()
            if not category_gas.empty:
                # Sort categories by gas usage for better visualization
                category_gas = category_gas.sort_values(by='gasCost', ascending=False)

                # Add percentage to the name
                total_gas = category_gas['gasCost'].sum()
                category_gas['name_with_pct'] = category_gas.apply(
                    lambda x: f"{x['opcode_category']} ({x['gasCost']/total_gas*100:.1f}%)",
                    axis=1
                )

                fig_cat_gas = px.pie(
                    category_gas,
                    values='gasCost',
                    names='name_with_pct',  # Use the new column with percentages
                    title=' '  # Empty title, we'll add it in layout
                )

                # Custom settings for better visualization
                fig_cat_gas.update_traces(
                    textposition='inside',
                    textinfo='percent',
                    insidetextorientation='radial'
                )

                # Position the title with extra space and make it bold
                fig_cat_gas.update_layout(
                    title={
                        'text': f'<b>Gas Cost by Opcode Category ({shorten_address(tx_hash)})</b>',  # Bold title
                        'y': 0.95,  # Position higher for more space
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 16}
                    },
                    margin=dict(t=100, b=50, l=50, r=50),  # Increased top margin for title
                )
                fig_cat_gas.show()
        except Exception as plot_err:
            console.print(f"[warning]Could not generate opcode category gas plot: {plot_err}", style="warning")

        # --- Plot Gas Cost per Opcode ---
        try:
            # Visulization Header
            console.print("\n\n[bold yellow3]⛽ Top 30 Opcodes by Gas Cost[/bold yellow3]")
            console.print("──────────────────────────────", style="yellow3")
            # Calculate opcode gas costs
            opcode_gas = df.groupby('op')['gasCost'].sum().sort_values(ascending=False).reset_index()
            # Only show opcodes with non-zero gas cost (as requested)
            opcode_gas = opcode_gas[opcode_gas['gasCost'] > 0]

            if not opcode_gas.empty:
                # Show all significant opcodes, not just top 30
                significant_opcodes = len(opcode_gas) if len(opcode_gas) <= 30 else 30

                fig_op_gas = px.bar(
                    opcode_gas.head(significant_opcodes),
                    x='op',
                    y='gasCost',
                    labels={'op': '<b>Opcode</b>', 'gasCost': '<b>Total Gas Cost</b>'}  # Bold axis labels
                )

                # Bold title
                fig_op_gas.update_layout(
                    title={
                        'text': f'<b>Top {significant_opcodes} Opcodes by Gas Cost ({shorten_address(tx_hash)})</b>',
                        'y': 0.95,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 16}
                    },
                    margin=dict(t=100, b=50, l=50, r=50)  # Increased top margin
                )
                fig_op_gas.show()
            else:
                 console.print("[info]No significant gas costs found per opcode.", style="info")
        except Exception as plot_err:
             console.print(f"[warning]Could not generate opcode gas plot: {plot_err}", style="warning")

        # --- Plot Gas Remaining Over Steps with PYUSD Highlighting ---
        try:
            # Downsample if too many steps to prevent browser freezing
            # ⚠️ WARNING: if there are too many steps it will consume lots of computes browser could crash or freeze.
            max_points_plot = 5000
            if len(df) > max_points_plot:
                # Ensure consistent sampling, e.g., take every Nth point
                indices = np.round(np.linspace(0, len(df) - 1, max_points_plot)).astype(int)
                plot_df = df.iloc[indices]
                plot_title = f'<b>Gas Remaining Over Execution Steps (Sampled) ({shorten_address(tx_hash)})</b>'
                xaxis_title = '<b>Execution Step (Sampled)</b>'
                console.print(
                    f"\n\n[bold yellow3]⛽ Plotting downsampled gas remaining[/bold yellow3] [info]({len(plot_df)} points out of {len(df)}).[/info]\n"
                    f"[yellow3]────────────────────────────────────────────────────────────────────[/yellow3]"
                )
            else:
                plot_df = df
                plot_title = f'<b>Gas Remaining Over Execution Steps ({shorten_address(tx_hash)})</b>'
                xaxis_title = '<b>Execution Step</b>'

            # Create a more informative plot with PYUSD sections highlighted
            fig_gas_steps = go.Figure()

            # Add base gas trace
            fig_gas_steps.add_trace(go.Scatter(
                x=plot_df['step'],
                y=plot_df['gas'],
                mode='lines',
                name='Gas Remaining',
                line=dict(color='blue')
            ))

            # Highlight PYUSD contract execution sections
            if 'is_pyusd_contract' in plot_df.columns:
                pyusd_sections = []
                current_section = None

                for i, row in plot_df.iterrows():
                    if row['is_pyusd_contract'] and current_section is None:
                        # Start a new PYUSD section
                        current_section = {'start': row['step']}
                    elif not row['is_pyusd_contract'] and current_section is not None:
                        # End the current PYUSD section
                        current_section['end'] = plot_df.iloc[i-1]['step'] if i > 0 else row['step']
                        pyusd_sections.append(current_section)
                        current_section = None

                # Handle case where the last section is a PYUSD section
                if current_section is not None:
                    current_section['end'] = plot_df.iloc[-1]['step']
                    pyusd_sections.append(current_section)

                # Add highlighted areas for PYUSD execution
                for section in pyusd_sections:
                    fig_gas_steps.add_shape(
                        type="rect",
                        x0=section['start'], x1=section['end'],
                        y0=0, y1=plot_df['gas'].max(),
                        fillcolor="rgba(0,255,0,0.1)",
                        line=dict(width=0),
                        layer="below"
                    )

            fig_gas_steps.update_layout(
                title={
                    'text': plot_title,
                    'y': 0.95,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {'size': 16}
                },
                xaxis_title=xaxis_title,
                yaxis_title='<b>Gas</b>',  # Bold y-axis label
                showlegend=True,
                margin=dict(t=100, b=60, l=60, r=60)  # Increased top margin
            )

            # Add annotation for PYUSD sections - MOVED TO BOTTOM LEFT
            if pyusd_percentage > 0:
                fig_gas_steps.add_annotation(
                    x=0.02, y=0.02,  # Bottom left
                    xref="paper", yref="paper",
                    text=f"Green sections: PYUSD contract execution ({pyusd_percentage:.1f}%)",
                    showarrow=False,
                    font=dict(color="green"),
                    bgcolor="white",
                    bordercolor="green",
                    borderwidth=1,
                    align="left"
                )

            fig_gas_steps.show()
        except Exception as plot_err:
             console.print(f"[warning]Could not generate gas remaining plot: {plot_err}", style="warning")

        # --- PYUSD-Specific Analysis ---
        if 'is_pyusd_contract' in df.columns and df['is_pyusd_contract'].any():
            console.print("\n[bold green3]🧩 PYUSD Contract Execution Analysis[/bold green3]")

            # Filter for PYUSD execution steps
            pyusd_df = df[df['is_pyusd_contract']]

            # Analyze PYUSD opcodes
            pyusd_opcodes = pyusd_df.groupby('op').size().sort_values(ascending=False)

            pyusd_table = Table(title="Top PYUSD Contract Operations", show_header=True, header_style="bold green3")
            pyusd_table.add_column("Opcode", style="dim")
            pyusd_table.add_column("Count", justify="right")
            pyusd_table.add_column("% of PYUSD Ops", justify="right")

            for op, count in pyusd_opcodes.head(10).items():
                percentage = (count / len(pyusd_df) * 100)
                pyusd_table.add_row(op, str(count), f"{percentage:.1f}%")

            console.print(pyusd_table)

            # Analyze PYUSD gas usage by category
            pyusd_gas_by_cat = pyusd_df.groupby('opcode_category')['gasCost'].sum().sort_values(ascending=False)

            try:
                # Create a pie chart for PYUSD gas usage by category
                gas_cat_df = pd.DataFrame({'category': pyusd_gas_by_cat.index, 'gas_used': pyusd_gas_by_cat.values})

                # Add percentage to the name
                total_gas = gas_cat_df['gas_used'].sum()
                gas_cat_df['name_with_pct'] = gas_cat_df.apply(
                    lambda x: f"{x['category']} ({x['gas_used']/total_gas*100:.1f}%)",
                    axis=1
                )

                fig_pyusd_gas = px.pie(
                    gas_cat_df,
                    values='gas_used',
                    names='name_with_pct',
                    title=' '  # Empty title, we'll add it in layout
                )

                # Custom settings for better visualization
                fig_pyusd_gas.update_traces(
                    textposition='inside',
                    textinfo='percent',
                    insidetextorientation='radial'
                )

                # Bold title
                fig_pyusd_gas.update_layout(
                    title={
                        'text': f'<b>PYUSD Contract Gas Usage by Category ({shorten_address(tx_hash)})</b>',
                        'y': 0.95,
                        'x': 0.5,
                        'xanchor': 'center',
                        'yanchor': 'top',
                        'font': {'size': 16}
                    },
                    margin=dict(t=100, b=50, l=50, r=50)  # Increased top margin
                )

                fig_pyusd_gas.show()
            except Exception as plot_err:
                console.print(f"[warning]Could not generate PYUSD gas category plot: {plot_err}", style="warning")

        # Display DataFrame sample first
        console.print("\n\n[bold cyan3]📊 structLog Data Sample (First 10 steps)[/bold cyan3]")
        console.print("──────────────────────────────────────────", style="cyan3")

        # Get a more informative view by focusing on key columns
        display_cols = ['step', 'op', 'opcode_category', 'gas', 'gasCost', 'depth', 'is_pyusd_contract']
        display(df[display_cols].head(10))

        console.print(f"[info]Full structLog DataFrame has {len(df)} rows. Displaying only head.", style="info")

        # PYUSD-specific summary
        pyusd_steps = df['is_pyusd_contract'].sum()
        if pyusd_steps > 0:
            pyusd_pct = (pyusd_steps / len(df)) * 100
            console.print(f"[success]PYUSD-specific execution: {pyusd_steps:,} steps ({pyusd_pct:.1f}% of total)", style="success")

            # Top Gas-Consuming PYUSD Operations
            if 'is_pyusd_contract' in df.columns:
                pyusd_ops_gas = df[df['is_pyusd_contract']].groupby('op')['gasCost'].sum().sort_values(ascending=False)

                if not pyusd_ops_gas.empty:
                    gas_table = Table(title="Top Gas-Consuming PYUSD Operations", show_header=True, header_style="bold green")
                    gas_table.add_column("Operation", style="dim")
                    gas_table.add_column("Gas Used", justify="right")
                    gas_table.add_column("% of PYUSD Gas", justify="right")

                    total_pyusd_gas = pyusd_ops_gas.sum()
                    for op, gas in pyusd_ops_gas.head(10).items():
                        percentage = (gas / total_pyusd_gas * 100)
                        gas_table.add_row(op, f"{gas:,}", f"{percentage:.1f}%")

                    console.print(gas_table)

        # Export options
        console.print("\n\n[bold cyan3]📤 Export Options:[/bold cyan3]")
        console.print("──────────────────", style="cyan3")

        # Create export output area
        export_output = widgets.Output()

        # Create export buttons with proper styling
        export_buttons = widgets.HBox([
            widgets.Button(
                description='Export to CSV',
                button_style='primary',  # Green
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export as JSON',
                button_style='warning',  # Orange
                layout=widgets.Layout(width='150px')
            ),
            widgets.Button(
                description='Export to Google Sheets',
                button_style='info',     # Blue
                layout=widgets.Layout(width='200px')
            )
        ])

        # Define export handlers
        def export_csv(b):
            with export_output:
                clear_output()
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"structlog_trace_{tx_hash[:10]}_{timestamp}.csv"
                display(download_csv_direct(df, filename))

        def export_json(b):
            with export_output:
                clear_output()
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"structlog_trace_{tx_hash[:10]}_{timestamp}.json"
                # Prepare export data with basic DataFrame stats to avoid reference issues
                export_data = {
                    "transaction_hash": tx_hash,
                    "analysis_type": "structLog",
                    "summary": {
                        "total_steps": len(df),
                        "total_gas_cost": total_gas_cost,
                        "max_depth": df['depth'].max() if not df.empty else None,
                        "max_stack_depth": df['stack_depth'].max() if not df.empty else None,
                        "max_memory_bytes": df['mem_size_bytes'].max() if not df.empty else None,
                        "pyusd_steps": pyusd_execution_steps,
                        "pyusd_percentage": pyusd_percentage
                    },
                    "opcode_categories": category_gas.to_dict('records') if 'category_gas' in locals() and not category_gas.empty else []
                }
                display(download_json_direct(export_data, filename))

        def export_to_sheets(b):
            with export_output:
                clear_output()
                try:
                    # Only use variables that are in scope and defined
                    export_data = {
                        "transaction_hash": tx_hash,
                        "summary": {
                            "total_steps": len(df),
                            "total_gas_cost": total_gas_cost,
                            "max_depth": df['depth'].max() if not df.empty else None,
                            "max_stack_depth": df['stack_depth'].max() if not df.empty else None,
                            "max_memory_bytes": df['mem_size_bytes'].max() if not df.empty else None,
                            "pyusd_steps": pyusd_execution_steps,
                            "pyusd_percentage": pyusd_percentage
                        }
                    }

                    # Add opcode categories if available
                    if 'category_gas' in locals() and not category_gas.empty:
                        export_data["opcode_categories"] = category_gas.to_dict('records')

                    display(export_to_google_sheets(df, export_data, tx_hash))
                except Exception as e:
                    html = f"<div style='color:red'>Error exporting to Google Sheets: {str(e)}</div>"
                    display(HTML(html))

                    # Fallback to CSV
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f"structlog_trace_{tx_hash[:10]}_{timestamp}.csv"
                    display(download_csv_direct(df, filename))
                    display(HTML("<div>Falling back to CSV download due to Google Sheets error.</div>"))

        # Connect handlers to buttons
        export_buttons.children[0].on_click(export_csv)
        export_buttons.children[1].on_click(export_json)
        export_buttons.children[2].on_click(export_to_sheets)

        # Display button container and output area
        display(export_buttons)
        display(export_output)

    return df


# --- Execute Tracing ---
# WARNING: structLog can be VERY large and slow. Default to False.
RUN_STRUCTLOG_TRACE = True  # <<< SET TO TRUE TO RUN THIS EXPENSIVE TRACE

if not validate_tx_hash:  # Check validation flag from setup cell
    console.print("[warning]TARGET_TX_HASH not set or invalid. Cannot run structLog analysis.", style="warning")
elif RUN_STRUCTLOG_TRACE:
    console.print("\n\n[bold]🎯 Using structLog on Mainnet[/bold]", style="cyan3")
    console.print("───────────────────────────────", style="cyan3")
    console.print(f"Target Transaction : {TARGET_TX_HASH}")

    # Use the default tracer configuration (the one that worked)
    tracer_config = {"tracerConfig": TRACE_CONFIGS["structLog"]}

    # Request trace on Mainnet
    trace_result_struct = make_rpc_request("debug_traceTransaction", [TARGET_TX_HASH, tracer_config], network='mainnet')

    # Process result if we got one
    if trace_result_struct:
        # Check where structLogs might be in the response
        struct_logs = None
        if 'structLogs' in trace_result_struct:
            struct_logs = trace_result_struct['structLogs']
        elif 'result' in trace_result_struct and isinstance(trace_result_struct['result'], dict) and 'structLogs' in trace_result_struct['result']:
            struct_logs = trace_result_struct['result']['structLogs']

        if struct_logs and isinstance(struct_logs, list):
            struct_log_df = parse_struct_log(struct_logs, TARGET_TX_HASH)
        else:
            console.print("[warning]Got response, but structLogs format was not as expected.", style="warning")
    else:
        console.print(f"[warning]Failed to get trace data for {TARGET_TX_HASH}.", style="warning")

elif not RUN_STRUCTLOG_TRACE:
     console.print("\n[info]Skipping trace analysis as RUN_STRUCTLOG_TRACE is False.", style="info")
